In [ ]:
pip install scikit-image

   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.8 MB 4.2 MB/s eta 0:00:03
   ------ --------------------------------- 2.1/12.8 MB 5.3 MB/s eta 0:00:03
   --------- ------------------------------ 3.1/12.8 MB 5.3 MB/s eta 0:00:02
   ------------- -------------------------- 4.2/12.8 MB 5.1 MB/s eta 0:00:02
   ---------------- ----------------------- 5.2/12.8 MB 5.0 MB/s eta 0:00:02
   --------------------- ------------------ 6.8/12.8 MB 5.4 MB/s eta 0:00:02
   -------------------------- ------------- 8.4/12.8 MB 5.8 MB/s eta 0:00:01
   ------------------------------- -------- 10.0/12.8 MB 5.8 MB/s eta 0:00:01
   ------------------------------------ --- 11.5/12.8 MB 6.1 MB/s eta 0:00:01
   ---------------------------------------  12.6/12.8 MB 6.0 MB/s eta 0:00:01
   ---------------------------------------- 12.8/12.8 MB 5.9 MB/s eta 0:00:00

   ---------------------------------------- 0/4 [tifffile]
   ----------------

In [ ]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions
from torch.distributions import kl, multivariate_normal
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import sys, random
import numpy as np
from numpy import asarray
import math
import collections
import pickle
from shutil import copyfile
import copy

import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import torchvision.transforms
from torchvision import datasets, models

import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, title, xlabel, ylabel, savefig, legend

from PIL import Image
import shutil, os
from os import listdir

from skimage import transform
from sklearn import metrics

from multiprocessing import Pool
import logging
import torch.multiprocessing as mp
import traceback
from contextlib import closing

import time
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, title, xlabel, ylabel, savefig, legend
from matplotlib import cm
from PIL import Image



reporttpraccuracy = True # True, False for attack, defence
# reporttpraccuracy = False
reporttprfscore = False # True, False for defence, attack
# reporttprfscore = True

sasearch = True # True for SA based linesearch

rlsearch = False # False for scalars based linesearch

attackmean=True

attackvar=True

savesaruns = False

uniformsampling = False

combinedattack = False


combinedsinglesattack = True

# klnorm = True
klnorm = True
vectornorm = False
controlledthreshold_sa=True
controlledthreshold_als=True
controlledthreshold_game=True

savemanipulations = False

mnistdataset=False

vggfacesdataset=True

oldversion=False

if (vggfacesdataset):
    testvae=True
    morelabels=False

    vggface_image_length=64

    layer_names = ['conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1',
                   'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2',
                   'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3', 'relu3_3', 'conv3_4', 'relu3_4', 'pool3',
                   'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3', 'relu4_3', 'conv4_4', 'relu4_4', 'pool4',
                   'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3', 'relu5_3', 'conv5_4', 'relu5_4', 'pool5']
    vae123_layers = ['relu1_1', 'relu2_1', 'relu3_1']
    vae345_layers = ['relu3_1', 'relu4_1', 'relu5_1']

    codesize = 61

    mylambda = 1.21
    mynumsteps = 10000003

    batcherror_threshold = 5
trainvae = False
traincnn = False

gamepreprocess = True
gamepostprocess = True

benchmarkalphas = False
benchmarkgasalphas = False
benchmarksasalphas = False


In [ ]:
def create_directory_structure():
    """Create the required directory structure for the project"""
    directories = [
        'data',
        'data/MNIST',
        'data/MNIST/raw',
        'data/manipulations',
        'data/manipulations/train',
        'data/manipulations/test',
        'data/originals',
        'data/originals/train',
        'data/originals/test',
        'data/vggfaces2',
        'data/manipulations',
        'data/manipulations/train',
        'data/manipulations/test',
        'data/originals',
        'data/originals/train',
        'data/originals/test'
    ]

    for directory in directories:
        os.makedirs(directory, exist_ok=True)
        print(f"Created directory: {directory}")


In [ ]:
 # Step 1: Create directory structure
create_directory_structure()
print()

Created directory: data
Created directory: data/MNIST
Created directory: data/MNIST/raw
Created directory: data/manipulations
Created directory: data/manipulations/train
Created directory: data/manipulations/test
Created directory: data/originals
Created directory: data/originals/train
Created directory: data/originals/test
Created directory: data/vggfaces2
Created directory: data/manipulations
Created directory: data/manipulations/train
Created directory: data/manipulations/test
Created directory: data/originals
Created directory: data/originals/train
Created directory: data/originals/test



In [ ]:
def download_mnist_pytorch():
    """Download MNIST using PyTorch's built-in functionality"""
    print("Downloading MNIST using PyTorch...")

    # This will download to '../data' relative to current directory
    # The code expects it in '../data' based on the datasets.MNIST('../data', ...) calls
    train_dataset = datasets.MNIST('../data', train=True, download=True,
                                   transform=transforms.Compose([
                                       transforms.ToTensor()
                                   ]))

    test_dataset = datasets.MNIST('../data', train=False, download=True,
                                  transform=transforms.Compose([
                                      transforms.ToTensor()
                                  ]))

    print(f"Training set size: {len(train_dataset)}")
    print(f"Test set size: {len(test_dataset)}")
    print("MNIST dataset downloaded successfully!")

    return train_dataset, test_dataset

In [ ]:
train_dataset, test_dataset = download_mnist_pytorch()
print()

100%|█████████████████████████████████████████████████████████████████████████████| 9.91M/9.91M [00:09<00:00, 1.06MB/s]
100%|█████████████████████████████████████████████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 56.7kB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 1.65M/1.65M [00:06<00:00, 239kB/s]
100%|█████████████████████████████████████████████████████████████████████████████| 4.54k/4.54k [00:00<00:00, 1.39MB/s]


Training set size: 60000
Test set size: 10000
MNIST dataset downloaded successfully!



In [ ]:
def preprocess_old_alphastar_mnist(alphastar):
    manip = torch.tensor(alphastar[:,:,0].reshape(1,32,32)).float()
    manip = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(28),
            transforms.ToTensor()
        ])(manip)
    return manip.clone()

def preprocess_old_alphastar_vggfaces2(alphastar):
    manip = torch.tensor(alphastar.reshape(3,32,32)).float()
    manip_max = torch.max(manip)
    manip_min = torch.min(manip)
    manip = ((manip - manip_min) / (manip_max - manip_min))*255.
    return manip.clone()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CDLLayer(nn.Module):
    def __init__(self, latent_dim, dict_size, tau=0.01, p=0.7, device=torch.device("cuda:0")):
        super(CDLLayer, self).__init__()
        self.latent_dim = latent_dim
        self.dict_size = dict_size
        self.tau = tau
        self.p = p
        self.device = device

        # Synthesis dictionary U (latent_dim x dict_size)
        self.U = nn.Parameter(torch.randn(latent_dim, dict_size, device=device))
        # Analysis dictionary V (latent_dim x dict_size)
        self.V = nn.Parameter(torch.randn(latent_dim, dict_size, device=device))

        self._initialize_dictionaries()

    def _initialize_dictionaries(self):
        # Initialize U with normalized columns
        with torch.no_grad():
            for i in range(self.dict_size):
                col_norm = torch.norm(self.U[:, i])
                if col_norm > 0:
                    self.U[:, i] = self.U[:, i] / col_norm

        # Initialize V randomly
        nn.init.xavier_normal_(self.V)

    def forward(self, z):
        """
        Apply CDL to latent representations: Z_recon = U * V^T * Z
        z: (batch_size, latent_dim)
        """
        batch_size = z.size(0)

        # Analysis: A = V^T * Z (dict_size x batch_size)
        VT_Z = torch.mm(self.V.t(), z.t())  # (dict_size, batch_size)

        # Synthesis: Z_recon = U * A (latent_dim x batch_size)
        z_reconstructed = torch.mm(self.U, VT_Z).t()  # (batch_size, latent_dim)

        return z_reconstructed, VT_Z.t()  # (batch_size, dict_size)

    def compute_cdl_loss(self, z):
        """
        Compute CDL reconstruction loss and sparsity regularization
        """
        batch_size = z.size(0)

        # Get reconstructed latent and coefficients
        z_recon, coeffs = self.forward(z)

        # Reconstruction loss: ||Z - U*V^T*Z||_F^2
        recon_loss = F.mse_loss(z_recon, z, reduction='mean')

        # Sparsity regularization on analysis dictionary V: ||V||_{2,p}
        row_norms = torch.norm(self.V, dim=1, p=2)  # L2 norm of each row

        if self.p == 0:
            # L_{2,0}: count of non-zero rows
            sparsity_loss = torch.sum((row_norms > 1e-8).float())
        else:
            # L_{2,p}: Σᵢ ||vᵢ||₂^p
            sparsity_loss = torch.sum(torch.pow(row_norms + 1e-8, self.p))

        # Total CDL loss
        cdl_loss = recon_loss + self.tau * sparsity_loss

        return cdl_loss, recon_loss, sparsity_loss

    def get_feature_importance(self):
        """
        Get feature importance based on row norms of analysis dictionary V
        """
        with torch.no_grad():
            return torch.norm(self.V, dim=1, p=2)

    def constraint_projection(self):
        """
        Project synthesis dictionary U to satisfy constraints: ||u_i||_2 <= 1
        """
        with torch.no_grad():
            for i in range(self.dict_size):
                col_norm = torch.norm(self.U[:, i])
                if col_norm > 1.0:
                    self.U[:, i] = self.U[:, i] / col_norm

    def update_U_admm(self, X, A, rho=1.0, max_iter=25):
        """
        Update synthesis dictionary U using ADMM
        Solve: min_U ||X - UA||²_F s.t. ||u_i||²_2 ≤ 1

        Args:
            X: Data matrix (latent_dim, n_samples)
            A: Coefficient matrix (dict_size, n_samples)
            rho: ADMM penalty parameter
            max_iter: Maximum iterations
        """
        d, k = self.U.shape
        n = A.shape[1]

        # ADMM variables
        U = self.U.data.clone()
        Z = self.U.data.clone()  # Auxiliary variable
        Lambda = torch.zeros_like(U, device=self.device)  # Dual variable

        # Precompute matrices for efficiency
        ATA = torch.mm(A, A.t())  # (dict_size, dict_size)
        XAT = torch.mm(X, A.t())  # (latent_dim, dict_size)
        eye_k = torch.eye(k, device=self.device)

        for it in range(max_iter):
            # Update U: solve (A*A^T + ρI)U = X*A^T + ρ(Z - Λ/ρ)
            lhs = ATA + rho * eye_k  # (dict_size, dict_size)
            rhs = XAT + rho * (Z - Lambda / rho)  # (latent_dim, dict_size)

            # Solve linear system: lhs^T * U^T = rhs^T
            try:
                U = torch.linalg.solve(lhs, rhs.t()).t()  # (latent_dim, dict_size)
            except RuntimeError:
                # Fallback to pseudo-inverse if singular
                U = torch.mm(rhs, torch.linalg.pinv(lhs))

            # Update Z with projection: ||z_i||_2 ≤ 1
            Z = U + Lambda / rho
            for i in range(k):
                col_norm = torch.norm(Z[:, i])
                if col_norm > 1.0:
                    Z[:, i] = Z[:, i] / col_norm

            # Update dual variable Λ
            Lambda = Lambda + rho * (U - Z)

            # Check convergence
            primal_residual = torch.norm(U - Z)
            if primal_residual < 1e-6:
                break

        # Update parameter
        self.U.data.copy_(U)

    def update_V_irls(self, X, max_iter=25):
        """
        Update analysis dictionary V using IRLS (Iteratively Reweighted Least Squares)
        Solve: min_V ||X - UV^TX||²_F + τ||V||²_{2,p}

        Args:
            X: Data matrix (latent_dim, n_samples)
            max_iter: Maximum iterations
        """
        d, k = self.V.shape
        n = X.shape[1]
        V = self.V.data.clone()
        eps = 1e-8  # For numerical stability

        # Precompute matrices
        XXT = torch.mm(X, X.t())  # (latent_dim, latent_dim)
        UUT = torch.mm(self.U.data, self.U.data.t())  # (latent_dim, latent_dim)
        XXT_UUT = torch.mm(XXT, UUT)  # (latent_dim, latent_dim)

        for it in range(max_iter):
            # Compute current row norms
            row_norms = torch.norm(V, dim=1, p=2) + eps  # Add epsilon for stability

            # Compute weights for reweighted least squares
            if self.p == 0:
                weights = torch.ones_like(row_norms)
            else:
                weights = self.p * torch.pow(row_norms, self.p - 2)

            # Create diagonal weight matrix W
            W = torch.diag(weights)

            # Solve reweighted problem:
            # (X*X^T*U*U^T + τW)V = X*X^T*U*U^T*V_old
            lhs = XXT_UUT + self.tau * W  # (latent_dim, latent_dim)
            rhs = torch.mm(XXT_UUT, V)    # (latent_dim, dict_size)

            # Solve linear system
            try:
                V_new = torch.linalg.solve(lhs, rhs)
            except RuntimeError:
                # Fallback to pseudo-inverse if singular
                V_new = torch.mm(torch.linalg.pinv(lhs), rhs)

            # Check convergence
            if torch.norm(V_new - V) < 1e-6:
                break

            V = V_new

        # Update parameter
        self.V.data.copy_(V)

    def cdl_optimization_step(self, X, max_outer_iter=10):
        """
        Perform one complete CDL optimization step with alternating updates

        Args:
            X: Data matrix (latent_dim, n_samples)
            max_outer_iter: Maximum outer iterations
        """
        X = X.t()  # Convert to (latent_dim, n_samples)

        for outer_it in range(max_outer_iter):
            # Step 1: Update A = V^T * X
            with torch.no_grad():
                A = torch.mm(self.V.data.t(), X)  # (dict_size, n_samples)

            # Step 2: Update U using ADMM
            self.update_U_admm(X, A)

            # Step 3: Update V using IRLS
            self.update_V_irls(X)

            # Apply constraints
            self.constraint_projection()

    def select_features(self, num_features):
        """
        Select top features based on analysis dictionary row norms

        Args:
            num_features: Number of features to select

        Returns:
            indices: Indices of selected features
            weights: Feature importance weights
        """
        feature_weights = self.get_feature_importance()
        _, indices = torch.topk(feature_weights, num_features)
        return indices, feature_weights[indices]

class Encoder(nn.Module):
    def __init__(self, nc, nef, nz, device):
        super(Encoder, self).__init__()
        self.device = device
        self.nz = nz

        # Encoder optimized for 64x64 input
        self.encoder = nn.Sequential(
            # 64x64 -> 32x32
            nn.Conv2d(nc, nef, 4, 2, padding=1),
            nn.LeakyReLU(0.2, True),
            nn.BatchNorm2d(nef),
            # 32x32 -> 16x16
            nn.Conv2d(nef, nef*2, 4, 2, padding=1),
            nn.LeakyReLU(0.2, True),
            nn.BatchNorm2d(nef*2),
            # 16x16 -> 8x8
            nn.Conv2d(nef*2, nef*4, 4, 2, padding=1),
            nn.LeakyReLU(0.2, True),
            nn.BatchNorm2d(nef*4),
            # 8x8 -> 4x4
            nn.Conv2d(nef*4, nef*8, 4, 2, padding=1),
            nn.LeakyReLU(0.2, True),
            nn.BatchNorm2d(nef*8)
        )

        # For 64x64 input, final feature map is 4x4
        self.mean = nn.Linear(nef*8*4*4, nz)
        self.logvar = nn.Linear(nef*8*4*4, nz)

    def forward(self, inputs):
        batch_size = inputs.size(0)
        # Encoded feature map (batch_size, nef*8, 4, 4)
        hidden = self.encoder(inputs)
        # Reshape to (batch_size, nef*8*4*4)
        hidden = hidden.view(batch_size, -1)
        mean, logvar = self.mean(hidden), self.logvar(hidden)
        return mean, logvar

class Decoder(nn.Module):
    def __init__(self, nc, ndf, nz):
        super(Decoder, self).__init__()
        self.ndf = ndf

        # Project latent to spatial feature map (4x4)
        self.latent_to_features = nn.Sequential(
            nn.Linear(nz, ndf*8*4*4),
            nn.ReLU(True)
        )

        # Progressive upsampling for 64x64 output
        self.decoder = nn.Sequential(
            # 4x4 -> 8x8
            nn.ConvTranspose2d(ndf * 8, ndf * 4, 4, 2, padding=1),
            nn.LeakyReLU(0.2, True),
            nn.BatchNorm2d(ndf * 4, 1.e-3),

            # 8x8 -> 16x16
            nn.ConvTranspose2d(ndf * 4, ndf * 2, 4, 2, padding=1),
            nn.LeakyReLU(0.2, True),
            nn.BatchNorm2d(ndf * 2, 1.e-3),

            # 16x16 -> 32x32
            nn.ConvTranspose2d(ndf * 2, ndf, 4, 2, padding=1),
            nn.LeakyReLU(0.2, True),
            nn.BatchNorm2d(ndf, 1.e-3),

            # 32x32 -> 64x64
            nn.ConvTranspose2d(ndf, nc, 4, 2, padding=1),
            nn.Tanh()  # Output in [-1, 1] range
        )

    def forward(self, input, target_size=None):
        batch_size = input.size(0)

        # Project to feature map
        x = self.latent_to_features(input)
        x = x.view(batch_size, self.ndf*8, 4, 4)

        # Decode to 64x64
        x = self.decoder(x)

        return x

class FLPLoss(nn.Module):
    def __init__(self, model_name, device, reduction='sum'):
        super(FLPLoss, self).__init__()
        self.device = device
        self.reduction = reduction
        # MSELoss
        self.loss_fn = nn.MSELoss(reduction=reduction)

    def forward(self, input, target):
        return self.loss_fn(input, target)

class KLDLoss(nn.Module):
    def __init__(self, reduction='sum'):
        super(KLDLoss, self).__init__()
        self.reduction = reduction

    def forward(self, mean, logvar):
        kld = -0.5 * torch.sum(1 + logvar - mean.pow(2) - logvar.exp(), dim=1)
        if self.reduction == 'sum':
            return torch.sum(kld)
        elif self.reduction == 'mean':
            return torch.mean(kld)
        else:
            return kld

In [ ]:
class VAE_VGGFACES2(nn.Module):
    def __init__(self, nc=3, ndf=32, nef=32, nz=128, device=torch.device("cuda:0"),
                 is_train=True, use_cdl=True, dict_size=512, cdl_tau=0.01, cdl_p=0.7):
        super(VAE_VGGFACES2, self).__init__()
        self.device = device
        self.nz = nz
        self.use_cdl = use_cdl

        # Encoder and Decoder for 64x64 images
        self.encoder = Encoder(nc=nc, nef=nef, nz=nz, device=device)
        self.decoder = Decoder(nc=nc, ndf=ndf, nz=nz)

        # CDL layer for latent space dictionary learning
        if self.use_cdl:
            self.cdl_layer = CDLLayer(nz, dict_size, cdl_tau, cdl_p, device)

        if is_train == False:
            for param in self.encoder.parameters():
                param.requires_grad = False
            for param in self.decoder.parameters():
                param.requires_grad = False
            if self.use_cdl:
                for param in self.cdl_layer.parameters():
                    param.requires_grad = False

    def encode(self, x):
        mean, logvar = self.encoder(x)
        return mean, logvar

    def decode(self, z, target_size=None):
        # target_size
        return self.decoder(z, target_size)

    def reparameterize(self, mean, logvar):
        std = logvar.mul(0.5).exp_()
        eps = torch.randn_like(std)
        return eps * std + mean

    def forward(self, x):
        # Ensure input is 64x64
        if x.size(2) != 64 or x.size(3) != 64:
            x = F.interpolate(x, size=(64, 64), mode='bilinear', align_corners=False)

        mean, logvar = self.encoder(x)
        latent_z = self.reparameterize(mean, logvar)

        # Apply CDL in latent space
        if self.use_cdl:
            latent_z_cdl, coeffs = self.cdl_layer(latent_z)
            rec_x = self.decoder(latent_z_cdl)
            return rec_x, mean, logvar, latent_z, coeffs
        else:
            rec_x = self.decoder(latent_z)
            return rec_x, mean, logvar


    def compute_cdl_loss(self, latent_z):
        """
        Compute CDL loss for the latent representations
        """
        if self.use_cdl:
            return self.cdl_layer.compute_cdl_loss(latent_z)
        else:
            return 0, 0, 0

    def apply_constraints(self):
        """
        Apply constraints to synthesis dictionary
        """
        if self.use_cdl:
            self.cdl_layer.constraint_projection()

    def get_feature_importance(self):
        """
        Get feature importance from CDL
        """
        if self.use_cdl:
            return self.cdl_layer.get_feature_importance()
        else:
            return None

    def perform_cdl_optimization(self, latent_z, max_iter=5):
        """
        Perform explicit CDL optimization step using ADMM/IRLS

        Args:
            latent_z: Latent representations (batch_size, latent_dim)
            max_iter: Maximum iterations for CDL optimization
        """
        if self.use_cdl:
            self.cdl_layer.cdl_optimization_step(latent_z, max_iter)


In [ ]:
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x.view(-1, 784), x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

def getlabeledindices(target, labels):
    labeledindices = torch.zeros(target.shape).byte()
    for label in labels:

        labeledindices = ((target == label) | labeledindices)

    return labeledindices


def get_parameters(model, bias=False):
    for k, m in model._modules.items():
        if k == "fc" and isinstance(m, nn.Linear):
            if bias:
                yield m.bias
            else:
                yield m.weight


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

    def forward(self, x):
        if (mnistdataset):
            x = F.relu(F.max_pool2d(self.conv1(x), 2))
            # print('x1',x.shape)
            x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
            # print('x2',x.shape)
            x = x.view(-1, 320)
            x = F.relu(self.fc1(x))
            # print('x6',x.shape)
            x = F.dropout(x, training=self.training)
            # print('x7',x.shape)
            x = self.fc2(x)
            # print('x8', x.shape)
        return F.log_softmax(x, dim=1) # Acrss CNN's, Make x shape (128,10)

# Use gpu for training cnn before and after game not during game?
def train(args, model, train_loader, optimizer, epoch):
    for batch_idx, (data, target) in enumerate(train_loader):

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        data, target = data.to(device, non_blocking=True), target.to(device, non_blocking=True)
        if (vggfacesdataset):
            data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        if (vggfacesdataset):
            criterion = nn.CrossEntropyLoss()
            loss = criterion(output, target)

        if (vggfacesdataset):
            loss.backward()
        optimizer.step()
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

# Use this loader to save manipulated images in VGGFace2 dataset. Need to redo preprocessing and training of CNN,VAE models.
def loader_vggfaces2(Rootdir, targetkey):
    images=[]
    labels=[]
    for img_file in listdir(Rootdir):
        img = Image.open(Rootdir + "/" + img_file)
        img = torchvision.transforms.Resize((vggface_image_length, vggface_image_length))(img)
        img = transforms.Compose([
            transforms.ToTensor()
        ])(img)
        images.append(img.clone())
        labels.append(float(targetkey))

    rootimages = torch.stack(images)
    rootlabels = torch.from_numpy(np.asarray(labels))

    # sys.exit()
    return (rootimages.clone(), rootlabels.clone())

In [ ]:
def test(args, model, test_loader, savemanipulations=True):

    testdata = []


    model.eval()

    targets = []
    predictions = []


    test_loss = 0
    correct = 0

    tps = {}
    ps = {}
    for argstarget in range(0, 10):
        tps[argstarget] = 0.
        ps[argstarget] = 0.

    tp = 0.
    p = 0.
    with torch.no_grad():
        batchidx = 0
        for data, target, origdata in test_loader:
            #data, target = data.to(device), target.to(device)

            output = model(data)
            if (vggfacesdataset):
                criterion = nn.CrossEntropyLoss()
                test_loss += criterion(output, target)

            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()


            argstarget = list(args['target'])[0]
            tp += torch.nonzero((target == argstarget).reshape(pred.shape) & (pred == argstarget)).size(0)
            p += torch.nonzero(target == argstarget).size(0)

            batcherror = 100. * (1.0 - (tp / p))
            print('batcherror=====================,savemanipulations',batcherror,savemanipulations)
            if(batcherror > batcherror_threshold):
                if (savemanipulations):
                    currtarget = target.view_as(pred)
                    incorrectidx = (pred != currtarget).reshape(-1)
                    print('incorrectidx.shape',incorrectidx.shape)

                    misclassifieddata = data[incorrectidx]
                    originaldata = origdata[incorrectidx]
                    misclassifiedtarget = currtarget[incorrectidx]

                    counter=0
                    for i in range(0, misclassifieddata.shape[0]):
                        advimg = misclassifieddata[i]
                        origimg = originaldata[i]
                        origclass = misclassifiedtarget[i].item()
                        predclass = pred[i].item()

                        print('origclass',origclass)
                        print('predclass',predclass)

                        if (origclass in args['positive_targets'] and predclass in args['negative_target']):
                            advimg = transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.Resize(256),
                                transforms.ToTensor()
                            ])(advimg)

                            origimg = transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.Resize(256),
                                transforms.ToTensor()
                            ])(origimg)

                            torchvision.utils.save_image(advimg, args['advtestimagessavepath'] + '/' + str(batchidx) + str(
                                i) + '_' + str(origclass) + ',' + str(predclass) + '.jpeg')
                            torchvision.utils.save_image(origimg,
                             args['origtestimagessavepath'] + '/' + str(batchidx) + str(
                                 i) + '_' + str(origclass) + ',' + str(predclass) + '.jpeg')
                            counter+=1

            targets.extend(target.numpy().tolist())
            predictions.extend(pred.view_as(target).numpy().tolist())

            testdata.extend(data)
            batchidx+=1

    test_loss /= len(test_loader.dataset)

    tpr =  tp/p

    tprs = {}

    if(reporttpraccuracy):
        return (100. * (1.0 - tpr), 100. * (1.0 - (correct / len(test_loader.dataset))), tprs)
    if(reporttprfscore):
        overallperf = 0.

        argstarget = list(args['target'])[0]

        overallperf += metrics.f1_score(targets, predictions, average=None, labels=[argstarget])[0]
        return (100. * (1.0 - tpr), 100. * (1.0 - overallperf), tprs)


In [ ]:
def adversarialmanipulation(posnegdata, posindexvector, negindexvector, encodedposwmean, encodedposwstddev,
                                deltawmean, deltawstddev, weightwmean, weightwstddev, autoencodermodel):

    # print('Entering adversarialmanipulation in maskedclassifiererror')
    advdata = torch.zeros(posnegdata.shape) # In generative model, encoded encodedposwmean need not have same shape as original posnegdata

    encodedadvdata = autoencodermodel.reparameterize(encodedposwmean + weightwmean,
                                                         encodedposwstddev + weightwstddev)
    advdata[posindexvector] = autoencodermodel.decode(encodedadvdata)
    advdata[negindexvector] = posnegdata[negindexvector]

    return advdata


In [ ]:
def maskedclassifiererror(posnegdata, posnegtargets, posindexvector, negindexvector, args, model, encodedposwmean, encodedposwstddev,
                          deltawmean, deltawstddev, weightwmean, weightwstddev, autoencodermodel, savemanipulations=True):


    advdata = adversarialmanipulation(posnegdata, posindexvector, negindexvector, encodedposwmean, encodedposwstddev,
                                      deltawmean, deltawstddev, weightwmean, weightwstddev, autoencodermodel)

    advtargets = posnegtargets

    adv_test_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(advdata, advtargets.long(), posnegdata),
        batch_size=args['test_batch_size'], shuffle=True)

    return test(args, model, adv_test_loader, savemanipulations)


In [ ]:
def linesearch_loss_function(posnegdata, advdata, autoencodermodel, args):

    adv_data_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(posnegdata, advdata),
        batch_size=args['batch_size'], shuffle=True)

    meanmu1batches = []
    meanmu2batches = []

    meanlogvar1batches = []
    meanlogvar2batches = []

    for posnegdatabatch, advdatabatch in adv_data_loader:
        mu1, logvar1 = autoencodermodel.encode(posnegdatabatch)
        mu2, logvar2 = autoencodermodel.encode(advdatabatch)

        batchmeanmu1 = mu1.mean(dim=0)
        batchmeanmu1 = batchmeanmu1.view(1, batchmeanmu1.shape[0])

        batchmeanmu2 = mu2.mean(dim=0)
        batchmeanmu2 = batchmeanmu2.view(1, batchmeanmu2.shape[0])

        batchmeanlogvar1 = logvar1.mean(dim=0)
        batchmeanlogvar1 = batchmeanlogvar1.view(1, batchmeanlogvar1.shape[0])

        batchmeanlogvar2 = logvar2.mean(dim=0)
        batchmeanlogvar2 = batchmeanlogvar2.view(1, batchmeanlogvar2.shape[0])

        meanmu1batches.append(batchmeanmu1.clone())
        meanmu2batches.append(batchmeanmu2.clone())

        meanlogvar1batches.append(batchmeanlogvar1.clone())
        meanlogvar2batches.append(batchmeanlogvar2.clone())


    mu1 = torch.stack(meanmu1batches)
    mu2 = torch.stack(meanmu2batches)
    logvar1 = torch.stack(meanlogvar1batches)
    logvar2 = torch.stack(meanlogvar2batches)

    meanmu1 = mu1.mean(dim=0)


    meanmu2 = mu2.mean(dim=0)

    meanlogvar1 = logvar1.mean(dim=0)

    meanlogvar2 = logvar2.mean(dim=0)

    mullogvar1 = torch.mul(meanlogvar1, torch.t(meanlogvar1))
    r1, c1 = mullogvar1.shape
    meanlogvar1 = torch.mul(mullogvar1,torch.eye(r1,c1))

    mullogvar2 = torch.mul(meanlogvar2, torch.t(meanlogvar2))
    r2, c2 = mullogvar2.shape
    meanlogvar2 = torch.mul(mullogvar2,torch.eye(r2,c2))

    advpdf = multivariate_normal.MultivariateNormal(meanmu2, meanlogvar2)
    posnegpdf = multivariate_normal.MultivariateNormal(meanmu1, meanlogvar1)

    KLD = kl.kl_divergence(advpdf,posnegpdf)

    if(torch.isnan(KLD)):
        print('Check KLD terms')

    return KLD

In [ ]:
def pidloop_test(x):
    return x*x

def pidloop(candidate_fitnessfnvalues,cid,pidx,candidate_weightwmean,candidate_weightwstddev,
            bestweightwmean,bestweightwstddev,deltawmean,deltawstddev,
            args,search_iter,weightwmean,weightwstddev,autoencodermodel,optimizemean,
            posnegdata,posnegtargets,posindexvector,negindexvector,model,
            encodedposwmean,encodedposwstddev):
    print('cid,pidx',cid,pidx)

    if(optimizemean):
        candidate_weightwmean[0, cid] = \
            bestweightwmean[0, cid] + \
            pidx * args['stepsizeweight'] * deltawmean[0, cid] * \
            math.pow(2.0 * args['stepsizeweight'], search_iter)

        print('Here3 1')

        candidate_intererror2, candidate_interoverallerror2, candidate_intertprserrors = \
            maskedclassifiererror(
                posnegdata,
                posnegtargets,
                posindexvector,
                negindexvector,
                args,
                model,
                encodedposwmean,
                encodedposwstddev,
                deltawmean,
                deltawstddev,
                candidate_weightwmean,
                weightwstddev,
                autoencodermodel)

        print('Here3 2')

        candidate_advdata = \
            adversarialmanipulation(
                posnegdata,
                posindexvector,
                negindexvector,
                deltawmean,
                encodedposwmean,
                encodedposwstddev,
                deltawstddev,
                candidate_weightwmean,
                weightwstddev,
                autoencodermodel)

        print('Here3 3')

        candidate_intermanipcost = linesearch_loss_function(posnegdata, candidate_advdata, autoencodermodel, args)
        candidate_intermanipcost = candidate_intermanipcost.item()
        candidate_fitnessfnvalue = (candidate_intererror2 - args['lambda'] * candidate_intermanipcost)

        print('Here3 4')

        candidate_fitnessfnvalues[pidx] = \
            (candidate_fitnessfnvalue,
             candidate_intererror2,
             candidate_intermanipcost,
             search_iter)

        print('Here3 final')
    else:
    # for pidx in pidxrange:
        candidate_weightwstddev[0, cid] = \
            bestweightwstddev[0, cid] + \
            pidx * args['stepsizeweight'] * deltawstddev[0, cid] * \
            math.pow(2.0 * args['stepsizeweight'], search_iter)

        candidate_intererror2, candidate_interoverallerror2, candidate_intertprserrors = \
            maskedclassifiererror(
                posnegdata,
                posnegtargets,
                posindexvector,
                negindexvector,
                args,
                model,
                encodedposwmean,
                encodedposwstddev,
                deltawmean,
                deltawstddev,
                weightwmean,
                candidate_weightwstddev,
                autoencodermodel)

        candidate_advdata = \
            adversarialmanipulation(
                posnegdata,
                posindexvector,
                negindexvector,
                deltawmean,
                encodedposwmean,
                encodedposwstddev,
                deltawstddev,
                weightwmean,
                candidate_weightwstddev,
                autoencodermodel)

        candidate_intermanipcost = linesearch_loss_function(posnegdata, candidate_advdata, autoencodermodel, args)
        candidate_intermanipcost = candidate_intermanipcost.item()
        candidate_fitnessfnvalue = (candidate_intererror2 - args['lambda'] * candidate_intermanipcost)

        candidate_fitnessfnvalues[pidx] = \
            (candidate_fitnessfnvalue,
             candidate_intererror2,
             candidate_intermanipcost,
             search_iter)

    return candidate_fitnessfnvalues

In [ ]:
def linsearchloop(cid, args, optimizemean, deltawmean, deltawstddev, weightwmean, weightwstddev, initialfitnessfnvalue, initialerror2, posnegdata, posnegtargets, posindexvector, negindexvector, model, encodedposwmean, encodedposwstddev, autoencodermodel, bestiterks, classkey, gameiter):

    print('Starting linsearchloop for cid,classkey, gameiter', cid,classkey, gameiter)

    bestweightwmean = weightwmean.clone()
    bestweightwstddev = weightwstddev.clone()

    print('args[maxiter], args[numsteps], args[mininc]', args['maxiter'], args['numsteps'], args['mininc'])

    if(optimizemean):
        candidate_weightwmean = weightwmean.clone()


        epsilon = (deltawmean[0, cid] / args['numsteps']).item()
        loopflag = True

        fitnessfnvalue = initialfitnessfnvalue  # Initialize fitnessfnvalue for greedy search
        error2 = initialerror2

        candidate_fitnessfnvalues = {}
        searchiter = 0
        bestpidx = 0

        best_x = bestpidxpayoff = bestpidxerror2 = bestpidxmanipcost = 0
        x = 0

        jumped = False

        while (loopflag and searchiter < args['maxiter']):
            x_step = epsilon * random.uniform(args['steplower'], args['stepupper'] - (args['stepdec'] * searchiter))
            x += x_step

            print('str(gameiter)+classkey, cid, searchiter, x, epsilon, deltawmean[0, cid]', str(gameiter) + ':' + classkey,
                  cid, searchiter, x, epsilon, deltawmean[0, cid])

            candidate_weightwmean[0, cid] += x

            candidate_intererror2, candidate_interoverallerror2, candidate_intertprserrors = \
                maskedclassifiererror(
                    posnegdata,
                    posnegtargets,
                    posindexvector,
                    negindexvector,
                    args,
                    model,
                    encodedposwmean,
                    encodedposwstddev,
                    deltawmean,
                    deltawstddev,
                    candidate_weightwmean,
                    weightwstddev,
                    autoencodermodel)

            if (klnorm):
                candidate_advdata = \
                    adversarialmanipulation(
                        posnegdata,
                        posindexvector,
                        negindexvector,
                        deltawmean,
                        encodedposwmean,
                        encodedposwstddev,
                        deltawstddev,
                        candidate_weightwmean,
                        weightwstddev,
                        autoencodermodel)

                candidate_intermanipcost = linesearch_loss_function(posnegdata, candidate_advdata, autoencodermodel,
                                                                    args)
            if (vectornorm):
                candidate_intermanipcost = torch.norm(candidate_weightwmean) + torch.norm(weightwstddev)

            candidate_intermanipcost = candidate_intermanipcost.item()
            candidate_fitnessfnvalue = (candidate_intererror2 - args['lambda'] * candidate_intermanipcost)

            candidate_fitnessfnvalues[searchiter] = \
                (x, candidate_fitnessfnvalue,
                 candidate_intererror2,
                 candidate_intermanipcost)

            fitnessfndiff = candidate_fitnessfnvalue - fitnessfnvalue

            errordiff = (candidate_intererror2 - error2)

            print('\n')
            print('x,x_step', x, x_step)
            print('candidate fitnessfnvalue', candidate_fitnessfnvalue)
            print('intermediate fitnessfnvalue', fitnessfnvalue)
            print('fitnessfndiff', fitnessfndiff)
            print('candidate error2', candidate_intererror2)
            print('intermediate error2', error2)
            print('errordiff', errordiff)
            print('candidate alphacost', candidate_intermanipcost)

            if (fitnessfndiff > 0):
                fitnessfnvalue = candidate_fitnessfnvalue
                error2 = candidate_intererror2
                bestpidx = searchiter

                (best_x, bestpidxpayoff,
                 bestpidxerror2,
                 bestpidxmanipcost) = \
                    (x, candidate_fitnessfnvalue,
                     candidate_intererror2,
                     candidate_intermanipcost)

                print('Invoking search')

                if (controlledthreshold_sa):
                    if (candidate_intererror2 > args['maxerror_sa']):
                        loopflag = False
                        print('Exiting sa for bestweightwmean with maxerror_sa,candidate_intererror2',args['maxerror_sa'],candidate_intererror2)
                        break
                    else:
                        loopflag = True
                else:
                    loopflag = True

            elif (errordiff > 0 and jumped == False):
                loopflag = True
                x += epsilon * random.uniform(args['steplower'], args['stepupper'])
                print('Invoking jump')
                jumped = True
            else:
                loopflag = False
                print('Exiting sa for bestweightwmean')

            searchiter += 1

            print('loopflag', loopflag)
            print('\n')

        bestweightwmean[0, cid] = candidate_weightwmean[0, cid]



        print('bestweightwmean[0, cid],cid', bestweightwmean[0, cid], cid)
        print('args[minpercent]', args['minpercent'])
        print('bestpidx', bestpidx)
        print('best_x', best_x)
        print('cid',cid)
        print('deltawmean[0, cid]', deltawmean[0, cid])
        print('\n')
    else:
        candidate_weightwstddev =  weightwstddev.clone()

        epsilon = (deltawstddev[0, cid] / args['numsteps']).item()

        loopflag = True

        fitnessfnvalue = initialfitnessfnvalue  # Initialize fitnessfnvalue for greedy search

        error2 = initialerror2

        candidate_fitnessfnvalues = {}
        searchiter = 0
        bestpidx = 0

        best_x = bestpidxpayoff = bestpidxerror2 = bestpidxmanipcost = 0
        x = 0

        jumped = False

        while (loopflag and searchiter < args['maxiter']):
            x_step = epsilon * random.uniform(args['steplower'], args['stepupper'] - (args['stepdec'] * searchiter))
            x += x_step

            print('str(gameiter)+classkey, cid, searchiter, x, epsilon, deltawstddev[0, cid]',
                  str(gameiter) + ':' + classkey, cid, searchiter, x, epsilon, deltawstddev[0, cid])

            # candidate_weightwstddev[0, cid] = bestweightwstddev[0, cid] + x
            candidate_weightwstddev[0, cid] += x

            candidate_intererror2, candidate_interoverallerror2, candidate_intertprserrors = \
                maskedclassifiererror(
                    posnegdata,
                    posnegtargets,
                    posindexvector,
                    negindexvector,
                    args,
                    model,
                    encodedposwmean,
                    encodedposwstddev,
                    deltawmean,
                    deltawstddev,
                    weightwmean,
                    candidate_weightwstddev,
                    autoencodermodel)

            if (klnorm):
                candidate_advdata = \
                    adversarialmanipulation(
                        posnegdata,
                        posindexvector,
                        negindexvector,
                        deltawmean,
                        encodedposwmean,
                        encodedposwstddev,
                        deltawstddev,
                        weightwmean,
                        candidate_weightwstddev,
                        autoencodermodel)

                candidate_intermanipcost = linesearch_loss_function(posnegdata, candidate_advdata, autoencodermodel,
                                                                    args)
            if (vectornorm):
                candidate_intermanipcost = torch.norm(weightwmean) + torch.norm(candidate_weightwstddev)

            candidate_intermanipcost = candidate_intermanipcost.item()
            candidate_fitnessfnvalue = (candidate_intererror2 - args['lambda'] * candidate_intermanipcost)

            candidate_fitnessfnvalues[searchiter] = \
                (x, candidate_fitnessfnvalue,
                 candidate_intererror2,
                 candidate_intermanipcost)

            fitnessfndiff = candidate_fitnessfnvalue - fitnessfnvalue
            # percentfitnessfndiff = ((candidate_fitnessfnvalue - fitnessfnvalue) / candidate_fitnessfnvalue) * 100

            errordiff = (candidate_intererror2 - error2)

            print('\n')
            print('x,x_step', x, x_step)
            print('candidate fitnessfnvalue', candidate_fitnessfnvalue)
            print('intermediate fitnessfnvalue', fitnessfnvalue)
            print('fitnessfndiff', fitnessfndiff)
            print('candidate error2', candidate_intererror2)
            print('intermediate error2', error2)
            print('errordiff', errordiff)
            print('candidate alphacost', candidate_intermanipcost)

            if (fitnessfndiff > 0):

                fitnessfnvalue = candidate_fitnessfnvalue
                error2 = candidate_intererror2
                bestpidx = searchiter

                (best_x, bestpidxpayoff,
                 bestpidxerror2,
                 bestpidxmanipcost) = \
                    (x, candidate_fitnessfnvalue,
                     candidate_intererror2,
                     candidate_intermanipcost)


                print('Invoking search')

                if (controlledthreshold_sa):
                    if (candidate_intererror2 > args['maxerror_sa']):
                        loopflag = False
                        print('Exiting sa for bestweightwmean with maxerror_sa,candidate_intererror2',args['maxerror_sa'],candidate_intererror2)
                        break
                    else:
                        loopflag = True
                else:
                    loopflag = True

            elif (errordiff > 0 and jumped == False):
                loopflag = True
                x += epsilon * random.uniform(args['steplower'], args['stepupper'])
                print('Invoking jump')
                jumped = True
            else:
                loopflag = False
                print('Exiting sa for bestweightwstddev')

            searchiter += 1

            print('loopflag', loopflag)
            print('\n')

        bestweightwstddev[0, cid] = candidate_weightwstddev[0, cid]

        print('bestweightwstddev[0, cid],cid', bestweightwstddev[0, cid], cid)
        print('args[minpercent]', args['minpercent'])
        print('bestpidx', bestpidx)
        print('best_x', best_x)
        print('cid',cid)
        print('deltawstddev[0, cid]', deltawstddev[0, cid])



    best_intererror2, best_interoverallerror2, best_intertprserrors = \
        maskedclassifiererror(
            posnegdata,
            posnegtargets,
            posindexvector,
            negindexvector,
            args,
            model,
            encodedposwmean,
            encodedposwstddev,
            deltawmean,
            deltawstddev,
            bestweightwmean,
            bestweightwstddev,
            autoencodermodel)

    if (klnorm):
        best_advdata = \
            adversarialmanipulation(
                posnegdata,
                posindexvector,
                negindexvector,
                deltawmean,
                encodedposwmean,
                encodedposwstddev,
                deltawstddev,
                bestweightwmean,
                bestweightwstddev,
                autoencodermodel)

        best_intermanipcost = linesearch_loss_function(posnegdata, best_advdata, autoencodermodel,
                                                       args)
    if(vectornorm):
        best_intermanipcost = torch.norm(bestweightwmean) + torch.norm(bestweightwstddev)




    best_intermanipcost = best_intermanipcost.item()
    best_fitnessfnvalue = (best_intererror2 - args['lambda'] * best_intermanipcost)

    print('best_intererror2,best_intermanipcost,best_fitnessfnvalue in linsearchloop', best_intererror2, best_intermanipcost,
          best_fitnessfnvalue)

    print('Ending linsearchloop for cid,classkey, gameiter', cid,classkey, gameiter)

    return (bestweightwmean, bestweightwstddev,bestiterks, best_fitnessfnvalue, best_intererror2)


In [ ]:
# Greedy Search or Simulated Annealing Search:
def linsearch(initialfitnessfnvalue, initialerror2, initialmanipcost, args, model, weightwmean, weightwstddev, optimizemean, posnegdata, posnegtargets, posindexvector, negindexvector,
              deltawmean, deltawstddev, encodedposwmean, encodedposwstddev, autoencodermodel, bestiterks, classkey, gameiter):

    if (savesaruns):
        fitnessfnvalues = torch.zeros(deltawmean.shape[1],args['maxiter'])
        error2s = torch.zeros(deltawmean.shape[1],args['maxiter'])
        manipcosts = torch.zeros(deltawmean.shape[1],args['maxiter'])

        weightwmeans = torch.zeros(deltawmean.shape[1],args['maxiter'],weightwmean.shape[0],weightwmean.shape[1])
        weightwstddevs = torch.zeros(deltawmean.shape[1],args['maxiter'],weightwmean.shape[0],weightwmean.shape[1])

    bestweightwmean = weightwmean.clone()
    bestweightwstddev = weightwstddev.clone()


    bestweightwmeans = {}
    bestweightwstddevs = {}

    print('args[maxiter], args[numsteps], args[mininc]', args['maxiter'], args['numsteps'], args['mininc'])
    print('---------------------------------------Starting SA loops--------------------------------------------')
    if(optimizemean):
        # for cid in xrange(0, 2):
        cids = range(0, deltawmean.shape[1])
        # random.shuffle(cids)
        completedcids = []
        sasattack=True
        cidsx=0
        while(sasattack and cidsx < len(cids)):

            cid = cids[cidsx]
            print('cid in optimizemean=True',cid)
            if (not combinedattack):
                candidate_weightwmean = weightwmean.clone()
            else:
                candidate_weightwmean = bestweightwmean.clone()


            epsilon = (deltawmean[0, cid] / args['numsteps']).item()
            print('epsilon',epsilon)

            loopflag = True

            fitnessfnvalue = initialfitnessfnvalue # Initialize fitnessfnvalue for greedy search

            error2 = initialerror2

            candidate_fitnessfnvalues = {}
            searchiter = 0
            bestpidx = 0

            best_x = bestpidxpayoff = bestpidxerror2 = bestpidxmanipcost = 0

            x = 0

            jumped=False

            while (loopflag and searchiter < args['maxiter']):

                x_step = epsilon * random.uniform(args['steplower'], args['stepupper'] - (args['stepdec'] * searchiter))
                x += x_step

                if (savesaruns):
                    fitnessfnvalues[cid, searchiter] = fitnessfnvalue
                    error2s[cid, searchiter] = error2
                    manipcosts[cid, searchiter] = initialmanipcost # Not using initialmanipcost in greedy search of generative model
                    weightwmeans[cid, searchiter, :, :] = bestweightwmean
                    weightwstddevs[cid, searchiter, :, :] = weightwstddev

                # x = x_range[idx_x]
                print('str(gameiter)+classkey, cid, searchiter, x, epsilon, deltawmean[0, cid]', str(gameiter) + ':' + classkey, cid, searchiter, x, epsilon, deltawmean[0, cid])

                # candidate_weightwmean[0, cid] = bestweightwmean[0, cid] + x
                candidate_weightwmean[0, cid] += x

                candidate_intererror2, candidate_interoverallerror2, candidate_intertprserrors = \
                    maskedclassifiererror(
                        posnegdata,
                        posnegtargets,
                        posindexvector,
                        negindexvector,
                        args,
                        model,
                        encodedposwmean,
                        encodedposwstddev,
                        deltawmean,
                        deltawstddev,
                        candidate_weightwmean,
                        weightwstddev,
                        autoencodermodel, True)

                if(klnorm):
                    candidate_advdata = \
                        adversarialmanipulation(
                            posnegdata,
                            posindexvector,
                            negindexvector,
                            deltawmean,
                            encodedposwmean,
                            encodedposwstddev,
                            deltawstddev,
                            candidate_weightwmean,
                            weightwstddev,
                            autoencodermodel)

                    candidate_intermanipcost = linesearch_loss_function(posnegdata, candidate_advdata, autoencodermodel,
                                                                        args)

                if(vectornorm):
                    candidate_intermanipcost = torch.norm(candidate_weightwmean) + torch.norm(weightwstddev)

                candidate_intermanipcost = candidate_intermanipcost.item()
                candidate_fitnessfnvalue = (candidate_intererror2 - args['lambda'] * candidate_intermanipcost)

                candidate_fitnessfnvalues[searchiter] = \
                    (x, candidate_fitnessfnvalue,
                     candidate_intererror2,
                     candidate_intermanipcost)

                fitnessfndiff = candidate_fitnessfnvalue - fitnessfnvalue

                errordiff = (candidate_intererror2 - error2)

                print('\n')

                print('cid, searchiter, x,x_step', cid, searchiter, x, x_step)
                print('candidate fitnessfnvalue', candidate_fitnessfnvalue)
                print('intermediate fitnessfnvalue', fitnessfnvalue)
                print('fitnessfndiff', fitnessfndiff)
                # print('percentfitnessfndiff', percentfitnessfndiff)
                print('candidate error2', candidate_intererror2)
                print('intermediate error2', error2)
                print('errordiff', errordiff)
                # print('percenterrordiff', percenterrordiff)
                print('candidate alphacost', candidate_intermanipcost)


                if (fitnessfndiff > 0):
                    fitnessfnvalue = candidate_fitnessfnvalue
                    error2 = candidate_intererror2
                    bestpidx = searchiter

                    (best_x, bestpidxpayoff,
                     bestpidxerror2,
                     bestpidxmanipcost) = \
                    (x, candidate_fitnessfnvalue,
                     candidate_intererror2,
                     candidate_intermanipcost)

                    print('Invoking search')

                    if (controlledthreshold_sa):
                        if (candidate_intererror2 > args['maxerror_sa']):
                            loopflag = False
                            print('Exiting sa for bestweightwmean with maxerror_sa,candidate_intererror2',args['maxerror_sa'],candidate_intererror2)
                            print('candidate_fitnessfnvalue, candidate_intererror2, candidate_intermanipcost', candidate_fitnessfnvalue, candidate_intererror2, candidate_intermanipcost)
                            break
                        else:
                            loopflag = True
                    else:
                        loopflag = True

                elif (errordiff > 0 and jumped == False):
                    loopflag = True
                    x += epsilon * random.uniform(args['steplower'], args['stepupper'])
                    print('Invoking jump')
                    jumped = True
                else:
                    loopflag = False
                    print('Exiting sa for bestweightwmean')

                searchiter+=1

                print('loopflag', loopflag)
                print('\n')

            if (not combinedattack):
                bestweightwmean = weightwmean.clone()
                bestweightwmean[0, cid] = candidate_weightwmean[0, cid]
            else:
                bestweightwmean = candidate_weightwmean.clone()


            best_intererror2, best_interoverallerror2, best_intertprserrors = \
                    maskedclassifiererror(
                        posnegdata,
                        posnegtargets,
                        posindexvector,
                        negindexvector,
                        args,
                        model,
                        encodedposwmean,
                        encodedposwstddev,
                        deltawmean,
                        deltawstddev,
                        bestweightwmean,
                        bestweightwstddev,
                        autoencodermodel)
            if (klnorm):
                best_advdata = \
                    adversarialmanipulation(
                        posnegdata,
                        posindexvector,
                        negindexvector,
                        deltawmean,
                        encodedposwmean,
                        encodedposwstddev,
                        deltawstddev,
                        bestweightwmean,
                        bestweightwstddev,
                        autoencodermodel)

                best_intermanipcost = linesearch_loss_function(posnegdata, best_advdata, autoencodermodel,
                                                                args)

            if (vectornorm):
                best_intermanipcost = torch.norm(bestweightwmean) + torch.norm(bestweightwstddev)

            best_intermanipcost = best_intermanipcost.item()
            best_fitnessfnvalue = (best_intererror2 - args['lambda'] * best_intermanipcost)

            print('\n')
            print('weightwmean[0, cid],cid', weightwmean[0, cid], cid)
            print('bestweightwmean[0, cid],cid', bestweightwmean[0, cid], cid)
            print('bestweightwmean[0, cid] + best_x,cid', bestweightwmean[0, cid] + best_x, cid)
            print('best_x', best_x)
            print('best_intererror2,best_intermanipcost,best_fitnessfnvalue',best_intererror2,best_intermanipcost,best_fitnessfnvalue)
            print('deltawmean[0, cid]', deltawmean[0, cid])
            print('args[minpercent]', args['minpercent'])
            print('bestpidx', bestpidx)
            print('classkey, gameiter',classkey, gameiter)
            print('\n')

            bestweightwmeans[cid] = (bestweightwmean,best_fitnessfnvalue,best_intererror2,best_intermanipcost)
            bestweightwstddevs[cid] = (bestweightwstddev,best_fitnessfnvalue,best_intererror2,best_intermanipcost)

            completedcids.append(cid)
            cidsx += 1


    else:
        cids = range(0, deltawstddev.shape[1])
        sasattack=True
        cidsx=0
        while(sasattack and cidsx < len(cids)):
            cid = cids[cidsx]
            print('cid in optimizemean=False',cid)
            if (not combinedattack):
                candidate_weightwstddev = weightwstddev.clone()
            else:
                candidate_weightwstddev = bestweightwstddev.clone()

            epsilon = (deltawstddev[0, cid] / args['numsteps']).item()
            loopflag = True

            fitnessfnvalue = initialfitnessfnvalue # Initialize fitnessfnvalue for greedy search

            error2 = initialerror2

            candidate_fitnessfnvalues = {}
            searchiter = 0
            bestpidx = 0

            best_x = bestpidxpayoff = bestpidxerror2 = bestpidxmanipcost = 0

            x = 0

            jumped=False

            while (loopflag and searchiter < args['maxiter']):

                x_step = epsilon * random.uniform(args['steplower'], args['stepupper'] - (args['stepdec'] * searchiter))
                x += x_step

                if (savesaruns):
                    fitnessfnvalues[cid, searchiter] = fitnessfnvalue
                    error2s[cid, searchiter] = error2
                    manipcosts[cid, searchiter] = initialmanipcost # Not using initialmanipcost in greedy search of generative model
                    weightwmeans[cid, searchiter, :, :] = weightwmean
                    weightwstddevs[cid, searchiter, :, :] = bestweightwstddev

                print('str(gameiter)+classkey, cid, searchiter, x, epsilon, deltawstddev[0, cid]', str(gameiter) + ':' + classkey, cid, searchiter, x, epsilon, deltawstddev[0, cid])
                candidate_weightwstddev[0, cid] += x

                candidate_intererror2, candidate_interoverallerror2, candidate_intertprserrors = \
                    maskedclassifiererror(
                        posnegdata,
                        posnegtargets,
                        posindexvector,
                        negindexvector,
                        args,
                        model,
                        encodedposwmean,
                        encodedposwstddev,
                        deltawmean,
                        deltawstddev,
                        weightwmean,
                        candidate_weightwstddev,
                        autoencodermodel)

                if (klnorm):
                    candidate_advdata = \
                        adversarialmanipulation(
                            posnegdata,
                            posindexvector,
                            negindexvector,
                            deltawmean,
                            encodedposwmean,
                            encodedposwstddev,
                            deltawstddev,
                            weightwmean,
                            candidate_weightwstddev,
                            autoencodermodel)

                    candidate_intermanipcost = linesearch_loss_function(posnegdata, candidate_advdata, autoencodermodel,
                                                                        args)

                if (vectornorm):
                    candidate_intermanipcost = torch.norm(weightwmean) + torch.norm(candidate_weightwstddev)

                candidate_intermanipcost = candidate_intermanipcost.item()
                candidate_fitnessfnvalue = (candidate_intererror2 - args['lambda'] * candidate_intermanipcost)

                candidate_fitnessfnvalues[searchiter] = \
                    (x, candidate_fitnessfnvalue,
                     candidate_intererror2,
                     candidate_intermanipcost)

                fitnessfndiff = candidate_fitnessfnvalue - fitnessfnvalue

                errordiff = (candidate_intererror2 - error2)
                print('\n')
                print('x,x_step', x,x_step)
                print('candidate fitnessfnvalue', candidate_fitnessfnvalue)
                print('intermediate fitnessfnvalue', fitnessfnvalue)
                print('fitnessfndiff', fitnessfndiff)
                # print('percentfitnessfndiff', percentfitnessfndiff)
                print('candidate error2', candidate_intererror2)
                print('intermediate error2', error2)
                print('errordiff', errordiff)
                # print('percenterrordiff', percenterrordiff)
                print('candidate alphacost', candidate_intermanipcost)

                if (fitnessfndiff > 0):
                    fitnessfnvalue = candidate_fitnessfnvalue
                    error2 = candidate_intererror2
                    bestpidx = searchiter

                    (best_x, bestpidxpayoff,
                     bestpidxerror2,
                     bestpidxmanipcost) = \
                    (x, candidate_fitnessfnvalue,
                     candidate_intererror2,
                     candidate_intermanipcost)

                    print('Invoking search')

                    if (controlledthreshold_sa):
                        if (candidate_intererror2 > args['maxerror_sa']):
                            loopflag = False
                            print('Exiting sa for bestweightwmean with maxerror_sa,candidate_intererror2',args['maxerror_sa'],candidate_intererror2)
                            break
                        else:
                            loopflag = True
                    else:
                        loopflag = True

                elif (errordiff > 0 and jumped == False):
                    loopflag = True
                    x += epsilon * random.uniform(args['steplower'], args['stepupper'])
                    print('Invoking jump')
                    jumped = True
                else:
                    loopflag = False
                    print('Exiting sa for bestweightwstddev')

                searchiter+=1

                print('loopflag', loopflag)
                print('\n')

            if (not combinedattack):
                bestweightwstddev = weightwstddev.clone()
                bestweightwstddev[0, cid] = candidate_weightwstddev[0, cid]
            else:
                bestweightwstddev = candidate_weightwstddev.clone()

            best_intererror2, best_interoverallerror2, best_intertprserrors = \
                    maskedclassifiererror(
                        posnegdata,
                        posnegtargets,
                        posindexvector,
                        negindexvector,
                        args,
                        model,
                        encodedposwmean,
                        encodedposwstddev,
                        deltawmean,
                        deltawstddev,
                        bestweightwmean,
                        bestweightwstddev,
                        autoencodermodel)
            if (klnorm):
                best_advdata = \
                    adversarialmanipulation(
                        posnegdata,
                        posindexvector,
                        negindexvector,
                        deltawmean,
                        encodedposwmean,
                        encodedposwstddev,
                        deltawstddev,
                        bestweightwmean,
                        bestweightwstddev,
                        autoencodermodel)

                best_intermanipcost = linesearch_loss_function(posnegdata, best_advdata, autoencodermodel,
                                                                args)

            if(vectornorm):
                best_intermanipcost = torch.norm(bestweightwmean) + torch.norm(bestweightwstddev)


            best_intermanipcost = best_intermanipcost.item()
            best_fitnessfnvalue = (best_intererror2 - args['lambda'] * best_intermanipcost)

            print('bestweightwstddev[0, cid],cid', bestweightwstddev[0, cid], cid)
            # print('percentfitnessfndiff', percentfitnessfndiff)
            print('args[minpercent]', args['minpercent'])
            # print('percentfitnessfndiff>args[minpercent]', percentfitnessfndiff > args['minpercent'])
            print('bestpidx', bestpidx)
            print('best_intererror2,best_intermanipcost,best_fitnessfnvalue',best_intererror2,best_intermanipcost,best_fitnessfnvalue)
            print('best_x', best_x)
            print('classkey, gameiter',classkey, gameiter)
            print('deltawstddev[0, cid]', deltawstddev[0, cid])
            print('\n')

            bestweightwstddevs[cid] = (bestweightwstddev,best_fitnessfnvalue,best_intererror2,best_intermanipcost)
            bestweightwmeans[cid] = (bestweightwmean,best_fitnessfnvalue,best_intererror2,best_intermanipcost)



            cidsx += 1


    print('---------------------------------------Ending SA loops--------------------------------------------')



    if(optimizemean):
        sortedbestweightwmeans = sorted(bestweightwmeans.items(), key=lambda kv: kv[1][1], reverse=True)
        bestweightwmeancid = sortedbestweightwmeans[0][0]
        bestweightwmean,bestwmeanpayoff,bestwmeanerror2,bestwmeanmanipcost = sortedbestweightwmeans[0][1]


        print('bestweightwmeancid,bestweightwmean,bestwmeanpayoff,bestwmeanerror2,bestwmeanmanipcost',
              bestweightwmeancid, bestweightwmean, bestwmeanpayoff,bestwmeanerror2,bestwmeanmanipcost)
    else:
        sortedbestweightwstddevs = sorted(bestweightwstddevs.items(), key=lambda kv: kv[1][1], reverse=True)
        bestweightwstddevcid = sortedbestweightwstddevs[0][0]
        bestweightwstddev,bestwstddevpayoff,bestwstddeverror2,bestwstddevmanipcost = sortedbestweightwstddevs[0][1]

        print('bestweightwstddevcid, bestweightwstddev,bestwstddevpayoff,bestwstddeverror2,bestwstddevmanipcost',
              bestweightwstddevcid, bestweightwstddev, bestwstddevpayoff,bestwstddeverror2,bestwstddevmanipcost)


    if (savesaruns):
        if(optimizemean):
            args['error2ssavepath'] = args['error2ssavepath'].rstrip('.pkl')+'mu'+'.pkl'
            args['manipcostssavepath'] = args['manipcostssavepath'].rstrip('.pkl')+'mu'+'.pkl'
            args['fitnessfnvaluessavepath'] = args['fitnessfnvaluessavepath'].rstrip('.pkl')+'mu'+'.pkl'

            args['weightwmeanssavepath'] = args['weightwmeanssavepath'].rstrip('.pkl')+'mu'+'.pkl'
            args['weightwstddevssavepath'] = args['weightwstddevssavepath'].rstrip('.pkl')+'mu'+'.pkl'

            error2sfp = open(args['error2ssavepath'], 'wb')
            torch.save(error2s, error2sfp)

            manipcostsfp = open(args['manipcostssavepath'], 'wb')
            torch.save(manipcosts, manipcostsfp)

            fitnessfnvaluesfp = open(args['fitnessfnvaluessavepath'], 'wb')
            torch.save(fitnessfnvalues, fitnessfnvaluesfp)

            weightwmeansfp = open(args['weightwmeanssavepath'], 'wb')
            torch.save(weightwmeans, weightwmeansfp)

            weightwstddevsfp = open(args['weightwstddevssavepath'], 'wb')


            torch.save(weightwstddevs, weightwstddevsfp)

            print('weightwmeansfp',args['weightwmeanssavepath'])
            print('weightwstddevssavepath',args['weightwstddevssavepath'])
            print('classkey', classkey)

            for cid in range(0, deltawmean.shape[1]):
                myx = weightwmeans[cid, :, :, cid].view(-1)
                myx = myx[myx.nonzero()].tolist()

                my1 = fitnessfnvalues[cid, :].view(-1)
                my1 = my1[my1.nonzero()].tolist()

                my2 = manipcosts[cid, :].view(-1)
                my2 = my2[my2.nonzero()].tolist()

                my3 = error2s[cid, :].view(-1)
                my3 = my3[my3.nonzero()].tolist()

                fig = plt.figure(1)
                ax = fig.add_subplot(111)
                ax.set_color_cycle(['blue', 'green', 'red'])

                ax.plot(myx, my1, label='payoffs', linestyle='-', marker='o', linewidth=2)
                ax.plot(myx, my2, label='costs', linestyle='--', marker='v', linewidth=2)
                ax.plot(myx, my3, label='errors', linestyle='-', marker='^', linewidth=2)

                xlabel('Step', fontsize=20)
                ylabel('Fitness', fontsize=20)

                handles, labels = ax.get_legend_handles_labels()
                lgd = ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.2), prop={'size': 20})

                ax.grid(linestyle='-', linewidth=0.4)
                ax.xaxis.set_tick_params(labelsize=20)
                ax.yaxis.set_tick_params(labelsize=20)

                fig.savefig(
                    "/home/achivuku/PycharmProjects/adversariallearning/saimages/" + str(gameiter) + ':' + classkey +'mu' + str(cid) + ".jpeg",
                    format='jpg', dpi=1200
                    # )
                    , bbox_extra_artists=(lgd,), bbox_inches='tight')
                plt.close()

            myx = weightwmeans[0,:,:,0].view(-1)
            myx = myx[myx.nonzero()].tolist()

            my1 = fitnessfnvalues[0, :].view(-1)


            my1 = my1[my1.nonzero()].tolist()

            my2 = manipcosts[0,:].view(-1)


            my2 = my2[my2.nonzero()].tolist()

            my3 = error2s[0,:].view(-1)

            my3 = my3[my3.nonzero()].tolist()

            print('myx',myx)
            print(weightwmeans[0,:,:,0].shape)
            print('my1',my1)
            print(fitnessfnvalues[0, :].shape)

            fig = plt.figure(1)
            ax = fig.add_subplot(111)
            ax.set_color_cycle(['blue', 'green', 'red'])

            ax.plot(myx, my1, label='payoffs', linestyle='-', marker='o', linewidth=2)
            ax.plot(myx, my2, label='costs', linestyle='--', marker='v', linewidth=2)
            ax.plot(myx, my3, label='errors', linestyle='-', marker='^', linewidth=2)

            xlabel('Step', fontsize=20)
            ylabel('Fitness', fontsize=20)

            handles, labels = ax.get_legend_handles_labels()
            lgd = ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.2), prop={'size': 20})

            ax.grid(linestyle='-', linewidth=0.4)
            ax.xaxis.set_tick_params(labelsize=20)
            ax.yaxis.set_tick_params(labelsize=20)

            fig.savefig(
                "/home/achivuku/PycharmProjects/adversariallearning/saimages/" + str(gameiter) + ':' + classkey + 'mu' + ".jpeg",
                format='jpg', dpi=1200
                # )
                , bbox_extra_artists=(lgd,), bbox_inches='tight')
            plt.close()

            print('Check payoff plots for gameiter:'+str(gameiter)+' ,classkey:'+classkey)
            # sys.exit()

        else:
            args['error2ssavepath'] = args['error2ssavepath'].rstrip('.pkl')+'var'+'.pkl'
            args['manipcostssavepath'] = args['manipcostssavepath'].rstrip('.pkl')+'var'+'.pkl'
            args['fitnessfnvaluessavepath'] = args['fitnessfnvaluessavepath'].rstrip('.pkl')+'var'+'.pkl'

            args['weightwmeanssavepath'] = args['weightwmeanssavepath'].rstrip('.pkl')+'var'+'.pkl'
            args['weightwstddevssavepath'] = args['weightwstddevssavepath'].rstrip('.pkl')+'var'+'.pkl'

            error2sfp = open(args['error2ssavepath'], 'wb')
            torch.save(error2s, error2sfp)

            manipcostsfp = open(args['manipcostssavepath'], 'wb')
            torch.save(manipcosts, manipcostsfp)

            fitnessfnvaluesfp = open(args['fitnessfnvaluessavepath'], 'wb')
            torch.save(fitnessfnvalues, fitnessfnvaluesfp)

            weightwmeansfp = open(args['weightwmeanssavepath'], 'wb')
            torch.save(weightwmeans, weightwmeansfp)

            weightwstddevsfp = open(args['weightwstddevssavepath'], 'wb')
            torch.save(weightwstddevs, weightwstddevsfp)

            print('weightwmeansfp',args['weightwmeanssavepath'])
            print('weightwstddevssavepath',args['weightwstddevssavepath'])
            print('classkey', classkey)

            for cid in xrange(0, deltawstddev.shape[1]):
                myx = weightwstddevs[cid, :, :, cid].view(-1)
                myx = myx[myx.nonzero()].tolist()

                my1 = fitnessfnvalues[cid, :].view(-1)
                my1 = my1[my1.nonzero()].tolist()

                my2 = manipcosts[cid, :].view(-1)
                my2 = my2[my2.nonzero()].tolist()

                my3 = error2s[cid, :].view(-1)
                my3 = my3[my3.nonzero()].tolist()

                fig = plt.figure(1)
                ax = fig.add_subplot(111)
                ax.set_color_cycle(['blue', 'green', 'red'])

                ax.plot(myx, my1, label='payoffs', linestyle='-', marker='o', linewidth=2)
                ax.plot(myx, my2, label='costs', linestyle='--', marker='v', linewidth=2)
                ax.plot(myx, my3, label='errors', linestyle='-', marker='^', linewidth=2)

                xlabel('Step', fontsize=20)
                ylabel('Fitness', fontsize=20)

                handles, labels = ax.get_legend_handles_labels()
                lgd = ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.2), prop={'size': 20})

                ax.grid(linestyle='-', linewidth=0.4)
                ax.xaxis.set_tick_params(labelsize=20)
                ax.yaxis.set_tick_params(labelsize=20)

                fig.savefig(
                    "/home/achivuku/PycharmProjects/adversariallearning/saimages/" + str(gameiter) + ':' + classkey + 'var' + str(cid) + ".jpeg",
                    format='jpg', dpi=1200
                    # )
                    , bbox_extra_artists=(lgd,), bbox_inches='tight')
                plt.close()

            myx = weightwstddevs[0,:,:,0].view(-1)

            # myx = weightwmeans[1,:,:,1].view(-1)
            myx = myx[myx.nonzero()].tolist()

            my1 = fitnessfnvalues[0, :].view(-1)
            # my1 = fitnessfnvalues[1, :].view(-1)
            my1 = my1[my1.nonzero()].tolist()

            my2 = manipcosts[0,:].view(-1)
            # my2 = manipcosts[1,:].view(-1)
            my2 = my2[my2.nonzero()].tolist()

            my3 = error2s[0,:].view(-1)
            # my3 = error2s[1,:].view(-1)
            my3 = my3[my3.nonzero()].tolist()

            print('myx',myx)
            print(weightwstddevs[0,:,:,0].shape)
            print('my1',my1)
            print(fitnessfnvalues[0, :].shape)


            fig = plt.figure(1)
            ax = fig.add_subplot(111)
            ax.set_color_cycle(['blue', 'green', 'red'])

            ax.plot(myx, my1, label='payoffs', linestyle='-', marker='o', linewidth=2)
            ax.plot(myx, my2, label='costs', linestyle='--', marker='v', linewidth=2)
            ax.plot(myx, my3, label='errors', linestyle='-', marker='^', linewidth=2)

            xlabel('Step', fontsize=20)
            ylabel('Fitness', fontsize=20)

            handles, labels = ax.get_legend_handles_labels()
            lgd = ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.2), prop={'size': 20})

            ax.grid(linestyle='-', linewidth=0.4)
            ax.xaxis.set_tick_params(labelsize=20)
            ax.yaxis.set_tick_params(labelsize=20)

            fig.savefig(
                "/home/achivuku/PycharmProjects/adversariallearning/saimages/" + str(gameiter) + ':' + classkey + 'var' + ".jpeg",
                format='jpg', dpi=1200
                # )
                , bbox_extra_artists=(lgd,), bbox_inches='tight')
            plt.close()


    if(optimizemean):
        return (bestweightwmeancid, bestweightwmean.clone(), bestweightwstddev.clone(), bestiterks, bestwmeanpayoff, bestwmeanerror2, bestwmeanmanipcost)
    else:
        return (bestweightwstddevcid, bestweightwmean.clone(), bestweightwstddev.clone(), bestiterks, bestwstddevpayoff, bestwstddeverror2, bestwstddevmanipcost)


In [ ]:
import sys
import os
from datetime import datetime
import traceback

class OutputCapture:
    def __init__(self, log_file_path):
        self.log_file = open(log_file_path, 'w', buffering=1)  # Line buffered
        self.original_stdout = sys.stdout
        self.original_stderr = sys.stderr

    def write(self, text):
        # Write to both console and file
        self.original_stdout.write(text)
        self.log_file.write(text)
        self.log_file.flush()  # Ensure immediate write to disk

    def flush(self):
        self.original_stdout.flush()
        self.log_file.flush()

    def close(self):
        self.log_file.close()

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
import torch
import torchvision.utils as vutils

def plot_learning_curves(train_losses, val_losses, title, filename):
    """Plot and save learning curves"""
    plt.figure(figsize=(12, 8))

    # Create subplots for different loss components
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # Total loss
    axes[0, 0].plot(train_losses['total'], label='Training Loss', color='blue')
    axes[0, 0].plot(val_losses['total'], label='Validation Loss', color='red')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Total Loss')
    axes[0, 0].set_title('Total Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True)

    # CDL loss
    axes[0, 1].plot(train_losses['cdl'], label='Training CDL Loss', color='green')
    axes[0, 1].plot(val_losses['cdl'], label='Validation CDL Loss', color='orange')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('CDL Loss')
    axes[0, 1].set_title('CDL Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True)

    # Reconstruction loss
    axes[1, 0].plot(train_losses['recon'], label='Training Recon Loss', color='purple')
    axes[1, 0].plot(val_losses['recon'], label='Validation Recon Loss', color='brown')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Reconstruction Loss')
    axes[1, 0].set_title('Reconstruction Loss')
    axes[1, 0].legend()
    axes[1, 0].grid(True)

    # KLD loss
    axes[1, 1].plot(train_losses['kld'], label='Training KLD Loss', color='pink')
    axes[1, 1].plot(val_losses['kld'], label='Validation KLD Loss', color='gray')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('KLD Loss')
    axes[1, 1].set_title('KLD Loss')
    axes[1, 1].legend()
    axes[1, 1].grid(True)

    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Learning curves saved to {filename}")

def save_reconstructed_and_original_images_cdl(model, traindata, traintargets, classes, num_images=20, device=torch.device('cpu'), save_dir='data/vae-cfm/cdl_vae_reconstructions/'):
    """
    Save original and reconstructed images for specified classes - CDL-VAE version
    """
    os.makedirs(save_dir, exist_ok=True)
    model.eval()

    all_original_images = []
    all_reconstructed_images = []

    with torch.no_grad():
        for cls in classes:
            print(f'Processing class {cls}...')

            cls_indices = (traintargets == cls).nonzero(as_tuple=True)[0]

            if len(cls_indices) < num_images:
                print(f'Warning: Only {len(cls_indices)} images available for class {cls}')
                selected_indices = cls_indices
            else:
                selected_indices = cls_indices[:num_images]

            class_original_images = []
            class_reconstructed_images = []

            for i, idx in enumerate(selected_indices):
                original_img = traindata[idx].unsqueeze(0).to(device)

                # Ensure image is 64x64
                if original_img.shape[-1] != 64 or original_img.shape[-2] != 64:
                    original_img = F.interpolate(original_img, size=(64, 64), mode='bilinear', align_corners=False)

                # Get reconstruction from CDL-VAE
                if model.use_cdl:
                    results = model(original_img)
                    if len(results) == 5:  # CDL format
                        rec_img, mean, logvar, latent_z, coeffs = results
                    else:  # Standard VAE
                        rec_img, mean, logvar = results
                else:
                    rec_img, mean, logvar = model(original_img)

                original_img = original_img.cpu()
                rec_img = rec_img.cpu()

                class_original_images.append(original_img)
                class_reconstructed_images.append(rec_img)

                # Save individual images
                vutils.save_image(original_img, f'{save_dir}/class_{cls}_original_{i}.png', normalize=True)
                vutils.save_image(rec_img, f'{save_dir}/class_{cls}_reconstructed_{i}.png', normalize=True)

            # Create class-wise comparison grids
            class_original_grid = torch.cat(class_original_images, dim=0)
            class_reconstructed_grid = torch.cat(class_reconstructed_images, dim=0)

            vutils.save_image(class_original_grid, f'{save_dir}/class_{cls}_original_grid.png',
                            nrow=5, normalize=True, padding=2)
            vutils.save_image(class_reconstructed_grid, f'{save_dir}/class_{cls}_reconstructed_grid.png',
                            nrow=5, normalize=True, padding=2)

            all_original_images.extend(class_original_images)
            all_reconstructed_images.extend(class_reconstructed_images)

            print(f'Saved {len(selected_indices)} images for class {cls}')

    # Create overall comparison
    comparison_images = []
    for orig, recon in zip(all_original_images, all_reconstructed_images):
        comparison_images.extend([orig, recon])

    comparison_tensor = torch.cat(comparison_images, dim=0)
    vutils.save_image(comparison_tensor, f'{save_dir}/all_classes_comparison.png',
                     nrow=10, normalize=True, padding=2)

    print(f'All images saved to {save_dir}')
    return all_original_images, all_reconstructed_images

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.utils import make_grid

def reconstruct_with_selected_features(model, data, indices, device):
    """
    Reconstruct images using only selected feature indices

    Args:
        model: Your trained CDL-VAE model
        data: Input batch of images [batch_size, channels, height, width]
        indices: Feature indices to keep (numpy array or tensor)
        device: torch device (cpu or cuda)

    Returns:
        reconstructed: Reconstructed images using only selected features
    """
    model.eval()
    with torch.no_grad():
        data = data.to(device)

        # Forward pass to get latent representation
        if model.use_cdl:
            _, mean, logvar, latent_z, _ = model(data)
        else:
            _, mean, logvar = model(data)
            latent_z = mean

        # Create mask for selected features
        mask = torch.zeros(latent_z.shape[1], device=device)
        if isinstance(indices, np.ndarray):
            indices = torch.from_numpy(indices).to(device)
        mask[indices] = 1.0

        # Apply mask to latent representation
        masked_latent = latent_z * mask.unsqueeze(0)

        # Decode masked latent vector
        reconstructed = model.decode(masked_latent)

    return reconstructed

def calculate_reconstruction_metrics(original, reconstructed):
    """
    Calculate various reconstruction error metrics

    Args:
        original: Original images tensor
        reconstructed: Reconstructed images tensor

    Returns:
        Dictionary of metrics
    """
    mse = F.mse_loss(reconstructed, original, reduction='mean').item()
    mae = F.l1_loss(reconstructed, original, reduction='mean').item()

    # PSNR calculation
    psnr = 20 * torch.log10(1.0 / torch.sqrt(F.mse_loss(reconstructed, original)))

    return {
        'MSE': mse,
        'MAE': mae,
        'PSNR': psnr.item()
    }

def plot_feature_comparison(original_data, recon_128, recon_64, recon_32,
                          metrics_128, metrics_64, metrics_32, num_imgs=6):
    """
    Plot comparison of reconstructions with different feature counts
    """
    fig, axes = plt.subplots(4, num_imgs, figsize=(3*num_imgs, 12))

    for i in range(num_imgs):
        # Original
        axes[0, i].imshow(original_data[i].cpu().permute(1, 2, 0).numpy().clip(0, 1))
        axes[0, i].set_title('Original')
        axes[0, i].axis('off')

        # 128 features
        axes[1, i].imshow(recon_128[i].cpu().permute(1, 2, 0).numpy().clip(0, 1))
        axes[1, i].set_title(f'128 features\nMSE: {metrics_128["MSE"]:.4f}')
        axes[1, i].axis('off')

        # 64 features
        axes[2, i].imshow(recon_64[i].cpu().permute(1, 2, 0).numpy().clip(0, 1))
        axes[2, i].set_title(f'64 features\nMSE: {metrics_64["MSE"]:.4f}')
        axes[2, i].axis('off')

        # 32 features
        axes[3, i].imshow(recon_32[i].cpu().permute(1, 2, 0).numpy().clip(0, 1))
        axes[3, i].set_title(f'32 features\nMSE: {metrics_32["MSE"]:.4f}')
        axes[3, i].axis('off')

    plt.tight_layout()
    plt.savefig('feature_comparison_reconstructions.png', dpi=300, bbox_inches='tight')
    plt.show()

# Main execution code
def compare_reconstructions_by_features(model, test_loader, device, results_dir='data/vae-cdl/vae-cdl_results'):
    """
    Compare reconstructions using different numbers of features
    """
    # Load feature indices
    indices_32 = np.load(f'{results_dir}/final_selected_features_32.npy')[:32]
    indices_64 = np.load(f'{results_dir}/final_selected_features_64.npy')[:64]
    indices_128 = np.arange(128)  # All features

    # Get a batch of test data
    data_iter = iter(test_loader)
    test_data, _ = next(data_iter)
    test_data = test_data[:8]  # Use first 8 images

    print("Performing reconstructions with different feature counts...")

    # Reconstruct with different feature counts
    with torch.no_grad():
        # Full reconstruction (128 features)
        model.eval()
        if model.use_cdl:
            recon_128, _, _, _, _ = model(test_data.to(device))
        else:
            recon_128, _, _ = model(test_data.to(device))

        # Selective reconstructions
        recon_64 = reconstruct_with_selected_features(model, test_data, indices_64, device)
        recon_32 = reconstruct_with_selected_features(model, test_data, indices_32, device)

    # Calculate metrics
    original_data = test_data.to(device)
    metrics_128 = calculate_reconstruction_metrics(original_data, recon_128)
    metrics_64 = calculate_reconstruction_metrics(original_data, recon_64)
    metrics_32 = calculate_reconstruction_metrics(original_data, recon_32)

    # Print results
    print("\n=== Reconstruction Quality Comparison ===")
    print(f"128 features: MSE={metrics_128['MSE']:.6f}, MAE={metrics_128['MAE']:.6f}, PSNR={metrics_128['PSNR']:.2f}dB")
    print(f"64 features:  MSE={metrics_64['MSE']:.6f}, MAE={metrics_64['MAE']:.6f}, PSNR={metrics_64['PSNR']:.2f}dB")
    print(f"32 features:  MSE={metrics_32['MSE']:.6f}, MAE={metrics_32['MAE']:.6f}, PSNR={metrics_32['PSNR']:.2f}dB")

    # Calculate quality retention
    quality_64 = 100 * (1 - (metrics_64['MSE'] - metrics_128['MSE']) / metrics_128['MSE'])
    quality_32 = 100 * (1 - (metrics_32['MSE'] - metrics_128['MSE']) / metrics_128['MSE'])

    print(f"\n=== Feature Efficiency ===")
    print(f"64 features retain {quality_64:.1f}% of 128-feature quality using 50% features")
    print(f"32 features retain {quality_32:.1f}% of 128-feature quality using 25% features")

    # Plot comparison
    plot_feature_comparison(test_data, recon_128, recon_64, recon_32,
                          metrics_128, metrics_64, metrics_32, num_imgs=6)

    return {
        'metrics_128': metrics_128,
        'metrics_64': metrics_64,
        'metrics_32': metrics_32,
        'quality_retention_64': quality_64,
        'quality_retention_32': quality_32
    }


In [ ]:
def playgame():
     # Create logs directory
    os.makedirs('logs', exist_ok=True)

    # Generate timestamp for unique log file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_file_path = f"logs/experiment_output_{timestamp}.log"

    # Set up output capture
    output_capture = OutputCapture(log_file_path)

    # Redirect stdout to capture all print statements
    sys.stdout = output_capture
    args = {}


    if(vggfacesdataset):
        args['batch_size'] = 64
        args['test_batch_size'] = 64
        args['lr'] = 1.0e-1
        args['momentum'] = 0.9
        args['weight_decay'] = 0.0
        args['step_size'] = 50
        args['gamma'] = 0.1
        args['epochs'] = 10
        criterion = nn.CrossEntropyLoss()

    args['log_interval'] = 10
    args['seed'] = 1

    if (vggfacesdataset):
        args['vae_num_epochs'] = 100
        args['model'] = "vae-123"
        args['initial_lr'] = 0.0005
        args['alpha'] = 1.0
        args['beta'] = 0.5


    args['cuda'] = torch.cuda.is_available()

    if (vggfacesdataset):
        positive_targets_ids = {}
        # positive_targets_ids["Andy_Lau"] = 1
        # positive_targets_ids["Zhang_Jingchu"] = 2
        positive_targets_ids["bkl"] = 0
        # positive_targets_ids["Aishwarya_Rai_Bachchan"] = 4
        # positive_targets_ids["Amy_Smart"] = 5
        # positive_targets_ids["Jada_Pinkett_Smith"] = 6
        # positive_targets_ids["Bruce_Willis"] = 7
        # positive_targets_ids["Oprah_Winfrey"] = 8
        # positive_targets_ids["Denzel_Washington"] = 9

        negative_targets_ids = {}
        negative_targets_ids["mel"] = 1
        # negative_targets_ids["Azim_Premji"] = 11
        negativeclasslabel = "mel"

        args['positive_targets'] = []
        for id in positive_targets_ids.values():
            args['positive_targets'].append(id)
        args['positive_targets'] = set(args['positive_targets'])
        args['negative_target'] = set(negative_targets_ids.values())



    if (vggfacesdataset):
        args['lambda'] = mylambda

        args['target'] = set([positive_targets_ids["bkl"]])
        positiveclasslabel = "bkl"
        positiveclasslabels = ["bkl"]

        if(trainvae):
            positiveclasslabels = ["bkl"]
        if(traincnn):
            positiveclasslabels = ["bkl"]
    if (vggfacesdataset):
        if(morelabels):
            additionalclasslabels = ["akiec", "bcc", "df", "nv", "vasc"]


    positiveskey = ''.join(map(str, args['positive_targets']))
    targetkey = ''.join(map(str, args['target']))
    negkey = ''.join(map(str, args['negative_target']))
    args['stepsizeweight'] = 1. / 100 # selected parameter value
    args['maxiter'] = 100

    args['numsteps'] = 20
    if (vggfacesdataset):
        args['numsteps'] = mynumsteps

    args['steplower'] = 1
    args['stepupper'] = 5
    args['stepdec'] = 0.5

    args['minpercent'] = 1
    args['mininc'] = 0.1
    args['maxerror_sa'] = 10
    args['maxerror_sa_combined'] = 10
    args['maxerror_als'] = 15
    args['maxerror_game'] = 15
    args['cdl_weight'] = 0.5
    args['cdl_update_freq'] = 15
    args['cdl_max_iter'] = 5
    args['feature_monitor_freq'] = 50
    args['num_selected_features'] = 100
    args['save_freq'] = 10
    args['results_dir'] = 'data/vae-cdl/vae-cdl_results'
    args['cdl_tau'] = 0.01
    args['cdl_p'] = 0.7
    args['dict_size'] = 512


    if (vggfacesdataset):
        args['vaetrainmodelsavepath'] = 'data/vae-cdl/initialvggfaces2vae-b3'+ str(codesize) +'.pth'

        args['traindatasetssavepath'] = 'data/vae-cdl/vggfaces2datasets-b3' + positiveskey + negkey + targetkey + '.pkl'
        args['trainlabelssavepath'] = 'data/vae-cdl/vggfaces2labels-b3' + positiveskey + negkey + targetkey + '.pkl'

        args['advtraindatasetssavepath'] = 'data/vae-cdl/vggfaces2advdatasets-b3'  + positiveskey + negkey + targetkey + '.pkl'
        args['advtrainlabelssavepath'] = 'data/vae-cdl/vggfaces2advlabels-b3'  + positiveskey + negkey + targetkey + '.pkl'

        args['advtestdatasetssavepath'] = 'data/vae-cdl/vggfaces2advtestdatasets-b3'  + positiveskey + negkey + targetkey + '.pkl'
        args['advtestlabelssavepath'] = 'data/vae-cdl/vggfaces2advtestlabels-b3'  + positiveskey + negkey + targetkey + '.pkl'

        # args['initialtrainmodelsavepath'] = '/data/achivuku/PycharmProjects/adversariallearning/vggfaces2/initialvggfaces2cnn-b3' + positiveskey + negkey + '.pth'
        args['initialtrainmodelsavepath'] = 'data/vae-cdl/initialvggfaces2cnn-b3' + positiveskey + negkey + targetkey + '.pth'


    args['advtrainimagessavepath'] = 'data/vae-cdl/manipulations/train'
    args['origtrainimagessavepath'] = 'data/vae-cdl/originals/train'

    args['advtestimagessavepath'] = 'data/vae-cdl/manipulations/test'
    args['origtestimagessavepath'] = 'data/vae-cdl/originals/test'


    if (not oldversion):

        # Uncomment this code block when experimenting with numsteps,lambda,codesize in cifar10
        if (vggfacesdataset):
            args['bestwmeanssavepath'] = 'data/vae-cdl/vggfaces2wmeanmanips-b3'  + targetkey + negkey + str(args['numsteps']) + str(args['lambda'])+ str(codesize) + '.pkl'
            args['bestwstddevssavepath'] = 'data/vae-cdl/vggfaces2wstddevmanips-b3'  + targetkey + negkey + str(args['numsteps']) + str(args['lambda'])+ str(codesize) + '.pkl'

            args['trainmodelsavepath'] = 'data/vae-cdl/vggfaces2cnn-b3' + targetkey + negkey  + str(args['numsteps']) + str(args['lambda']) + str(codesize) + '.pth'
            args['finaltrainmodelsavepath'] = 'data/vae-cdl/finalvggfaces2cnn-b3' + targetkey + negkey + str(args['numsteps']) + str(args['lambda']) + str(codesize) + '.pth'



    if (vggfacesdataset):
        negative_train = "ham10000/classified_images_train/" + negativeclasslabel
        Rootdir = negative_train
        negtrainimages, negtrainlabels = loader_vggfaces2(Rootdir, negkey)

        vggfacestraindata = negtrainimages.clone()
        vggfacestraintargets = negtrainlabels.clone()

        vggfacespostraindata = torch.zeros(0, 3, vggface_image_length, vggface_image_length)
        vggfacespostraintargets = torch.zeros(0, )

        vggfacesnegtraindata = negtrainimages.clone()
        vggfacesnegtraintargets = negtrainlabels.clone()

        for allpositiveclasslabel in positiveclasslabels:
            positive_train = "ham10000/classified_images_train/" + allpositiveclasslabel

            Rootdir = positive_train
            postrainimages, postrainlabels = loader_vggfaces2(Rootdir, targetkey)

            vggfacestraindata = torch.cat((vggfacestraindata, postrainimages), 0)
            vggfacestraintargets = torch.cat((vggfacestraintargets, postrainlabels), 0)


            vggfacespostraindata = torch.cat((vggfacespostraindata, postrainimages), 0)
            vggfacespostraintargets = torch.cat((vggfacespostraintargets, postrainlabels.float()), 0)


        positive_test = "ham10000/classified_images_val/" + positiveclasslabel
        negative_test = "ham10000/classified_images_val/" + negativeclasslabel

        Rootdir = positive_test
        postestimages, postestlabels = loader_vggfaces2(Rootdir, targetkey)
        Rootdir = negative_test
        negtestimages, negtestlabels = loader_vggfaces2(Rootdir, negkey)

        vggfacestestdata = torch.cat((postestimages, negtestimages), 0)
        vggfacestesttargets = torch.cat((postestlabels, negtestlabels), 0)

        train_loader = torch.utils.data.DataLoader(
            torch.utils.data.TensorDataset(vggfacestraindata, vggfacestraintargets.long()),
            batch_size=args['batch_size'], shuffle=True, num_workers=4)

        test_loader = torch.utils.data.DataLoader(
            torch.utils.data.TensorDataset(vggfacestestdata, vggfacestesttargets.long()),
            batch_size=args['test_batch_size'], shuffle=True, num_workers=4)


    # vaepreprocess = True
    vaepreprocess = True # No need to preprocess VAE data until entire MNIST database is used
    if(vaepreprocess):
        classpartitioneddatasets = collections.defaultdict()
        classpartitionedlabels = collections.defaultdict()

        if (vggfacesdataset):
            classpartitioneddatasets[targetkey] = vggfacespostraindata.clone()
            classpartitionedlabels[targetkey] = vggfacespostraintargets.clone()

            classpartitioneddatasets[negkey] = vggfacesnegtraindata.clone()
            classpartitionedlabels[negkey] = vggfacesnegtraintargets.clone()


        datasetsfp = open(args['traindatasetssavepath'],'wb')
        labelsfp = open(args['trainlabelssavepath'],'wb')
        pickle.dump(classpartitioneddatasets,datasetsfp)
        pickle.dump(classpartitionedlabels,labelsfp)
        print('Data preprocessing done !')
        # sys.exit()

    datasetsfp = open(args['traindatasetssavepath'], 'rb')
    labelsfp = open(args['trainlabelssavepath'], 'rb')
    traindatasets = pickle.load(datasetsfp)
    trainlabels = pickle.load(labelsfp)

    if (vggfacesdataset):
        # traindata = torch.zeros(0, 3, 28, 28)
        traindata = torch.zeros(0, 3, vggface_image_length, vggface_image_length)

    traintargets = torch.zeros(0, )
    for key in traindatasets.keys():
        data, target = traindatasets[key], trainlabels[key].float()
        print('data.shape','target.shape',data.shape,target.shape)
        print('traindata.shape','traintargets.shape',traindata.shape,traintargets.shape)
        traindata, traintargets = torch.cat((traindata,data),0), torch.cat((traintargets,target),0)

    # sys.exit()


    if(trainvae):
        device = torch.device("cuda" if args['cuda'] else "cpu")

        # Initialize loss tracking
        train_losses = {
            'total': [],
            'kld': [],
            'recon': [],
            'cdl': [],
            'cdl_recon': [],
            'cdl_sparsity': []
        }
        val_losses = {
            'total': [],
            'kld': [],
            'recon': [],
            'cdl': [],
            'cdl_recon': [],
            'cdl_sparsity': []
        }

        if (vggfacesdataset):
            # Initialize model with CDL parameters
            model = VAE_VGGFACES2(
                nc=3,                                    # RGB channels
                ndf=32,                                  # Decoder feature dimension
                nef=32,                                  # Encoder feature dimension
                nz=128,                                  # Latent dimension
                device=device,
                is_train=True,                           # Training mode
                use_cdl=True,                           # Enable CDL
                dict_size=args.get('dict_size', 256),   # Dictionary size
                cdl_tau=args.get('cdl_tau', 0.1),       # CDL sparsity weight
                cdl_p=args.get('cdl_p', 0.5)            # CDL regularization parameter
            ).to(device)

            # Reconstruction loss
            reconst_criterion = FLPLoss(args['model'], device, reduction='sum')
            # KLD loss
            kld_criterion = KLDLoss(reduction='sum')
            optimizer = optim.Adam(model.parameters(), lr=args['initial_lr'])
            model.train(True)

        for epoch in range(args['vae_num_epochs']):
            # TRAINING PHASE
            model.train()
            epoch_loss = 0
            epoch_kld = 0
            epoch_recon = 0
            epoch_cdl = 0
            epoch_cdl_recon = 0
            epoch_cdl_sparsity = 0

            for batch_idx, (data, target) in enumerate(train_loader):
                if (vggfacesdataset):
                    data = data.to(device)

                    # Ensure data is 64x64 (the model will auto-resize if needed)
                    if data.size(2) != 64 or data.size(3) != 64:
                        data = F.interpolate(data, size=(64, 64), mode='bilinear', align_corners=False)

                optimizer.zero_grad()

                if (vggfacesdataset):
                    # Forward pass - handle CDL outputs
                    if model.use_cdl and model.training:
                        rec_data, mean, logvar, latent_z, coeffs = model(data)
                        # Compute CDL loss components
                        cdl_loss, cdl_recon_loss, cdl_sparsity_loss = model.compute_cdl_loss(latent_z)
                    else:
                        rec_data, mean, logvar = model(data)
                        cdl_loss = 0
                        cdl_recon_loss = 0
                        cdl_sparsity_loss = 0

                    # Compute standard losses
                    reconst_loss = reconst_criterion(rec_data, data)
                    kld_loss = kld_criterion(mean, logvar)

                    # Total loss with CDL component
                    loss = (args["alpha"] * kld_loss +
                           args["beta"] * reconst_loss +
                           args.get("cdl_weight", 1.0) * cdl_loss)

                    # Accumulate losses for logging
                    epoch_loss += loss.item()
                    epoch_kld += kld_loss.item()
                    epoch_recon += reconst_loss.item()

                    # Handle CDL losses (check if tensor or scalar)
                    if isinstance(cdl_loss, torch.Tensor):
                        epoch_cdl += cdl_loss.item()
                        epoch_cdl_recon += cdl_recon_loss.item()
                        epoch_cdl_sparsity += cdl_sparsity_loss.item()
                    else:
                        epoch_cdl += cdl_loss
                        epoch_cdl_recon += cdl_recon_loss
                        epoch_cdl_sparsity += cdl_sparsity_loss

                # Standard gradient-based optimization
                loss.backward()
                optimizer.step()

                # Apply dictionary constraints after parameter update
                if model.use_cdl:
                    model.apply_constraints()

                # **NEW: Explicit CDL optimization using ADMM/IRLS every N batches**
                if (model.use_cdl and model.training and
                    batch_idx % args.get('cdl_update_freq', 10) == 0):

                    with torch.no_grad():
                        # Perform explicit CDL optimization step
                        model.perform_cdl_optimization(latent_z,
                                                     max_iter=args.get('cdl_max_iter', 3))

                # **NEW: Optional feature selection monitoring every M batches**
                if (model.use_cdl and
                    batch_idx % args.get('feature_monitor_freq', 50) == 0):

                    with torch.no_grad():
                        feature_importance = model.get_feature_importance()
                        num_active_features = torch.sum(feature_importance > 1e-6).item()
                        avg_importance = torch.mean(feature_importance).item()

                        if batch_idx % 100 == 0:  # Print less frequently
                            print(f'Batch {batch_idx}: Active features: {num_active_features}/{len(feature_importance)}, '
                                  f'Avg importance: {avg_importance:.6f}')

            # VALIDATION PHASE
            model.eval()
            val_epoch_loss = 0
            val_epoch_kld = 0
            val_epoch_recon = 0
            val_epoch_cdl = 0
            val_epoch_cdl_recon = 0
            val_epoch_cdl_sparsity = 0
            val_batches = 0

            with torch.no_grad():
                for val_data, val_target in test_loader:
                    if vggfacesdataset:
                        val_data = val_data.to(device)
                        if val_data.size(2) != 64 or val_data.size(3) != 64:
                            val_data = F.interpolate(val_data, size=(64, 64), mode='bilinear', align_corners=False)

                    if vggfacesdataset:
                        # Forward pass - handle outputs consistently
                        val_results = model(val_data)

                        if model.use_cdl:
                            # CDL enabled: expect 5 outputs
                            if len(val_results) == 5:
                                val_rec_data, val_mean, val_logvar, val_latent_z, val_coeffs = val_results
                                val_cdl_loss, val_cdl_recon_loss, val_cdl_sparsity_loss = model.compute_cdl_loss(val_latent_z)
                            else:
                                raise ValueError(f"Expected 5 outputs with CDL enabled, got {len(val_results)}")
                        else:
                            # CDL disabled: expect 3 outputs
                            if len(val_results) == 3:
                                val_rec_data, val_mean, val_logvar = val_results
                                val_cdl_loss = 0
                                val_cdl_recon_loss = 0
                                val_cdl_sparsity_loss = 0
                            else:
                                raise ValueError(f"Expected 3 outputs with CDL disabled, got {len(val_results)}")

                        # Compute validation losses
                        val_reconst_loss = reconst_criterion(val_rec_data, val_data)
                        val_kld_loss = kld_criterion(val_mean, val_logvar)

                        val_loss = (args["alpha"] * val_kld_loss +
                                   args["beta"] * val_reconst_loss +
                                   args.get("cdl_weight", 1.0) * val_cdl_loss)

                        # Accumulate validation losses
                        val_epoch_loss += val_loss.item()
                        val_epoch_kld += val_kld_loss.item()
                        val_epoch_recon += val_reconst_loss.item()

                        if isinstance(val_cdl_loss, torch.Tensor):
                            val_epoch_cdl += val_cdl_loss.item()
                            val_epoch_cdl_recon += val_cdl_recon_loss.item()
                            val_epoch_cdl_sparsity += val_cdl_sparsity_loss.item()
                        else:
                            val_epoch_cdl += val_cdl_loss
                            val_epoch_cdl_recon += val_cdl_recon_loss
                            val_epoch_cdl_sparsity += val_cdl_sparsity_loss

                    val_batches += 1

            # Store losses for plotting
            num_train_batches = len(train_loader)
            num_val_batches = val_batches

            train_losses['total'].append(epoch_loss / num_train_batches)
            train_losses['kld'].append(epoch_kld / num_train_batches)
            train_losses['recon'].append(epoch_recon / num_train_batches)
            train_losses['cdl'].append(epoch_cdl / num_train_batches)
            train_losses['cdl_recon'].append(epoch_cdl_recon / num_train_batches)
            train_losses['cdl_sparsity'].append(epoch_cdl_sparsity / num_train_batches)

            val_losses['total'].append(val_epoch_loss / num_val_batches)
            val_losses['kld'].append(val_epoch_kld / num_val_batches)
            val_losses['recon'].append(val_epoch_recon / num_val_batches)
            val_losses['cdl'].append(val_epoch_cdl / num_val_batches)
            val_losses['cdl_recon'].append(val_epoch_cdl_recon / num_val_batches)
            val_losses['cdl_sparsity'].append(val_epoch_cdl_sparsity / num_val_batches)

            # Print epoch statistics
            print('=== Epoch [{}/{}] Summary ==='.format(epoch + 1, args['vae_num_epochs']))
            print('  TRAINING:')
            print('    Avg Total Loss: {:.4f}, KLD: {:.4f}, Recon: {:.4f}, CDL: {:.4f}, CDL_Recon: {:.4f}, CDL_Sparse: {:.4f}'
                  .format(train_losses['total'][-1], train_losses['kld'][-1], train_losses['recon'][-1],
                         train_losses['cdl'][-1], train_losses['cdl_recon'][-1], train_losses['cdl_sparsity'][-1]))
            print('  VALIDATION:')
            print('    Avg Total Loss: {:.4f}, KLD: {:.4f}, Recon: {:.4f}, CDL: {:.4f}, CDL_Recon: {:.4f}, CDL_Sparse: {:.4f}'
                  .format(val_losses['total'][-1], val_losses['kld'][-1], val_losses['recon'][-1],
                         val_losses['cdl'][-1], val_losses['cdl_recon'][-1], val_losses['cdl_sparsity'][-1]))
            print('=' * 40)

            # **NEW: Save feature importance at end of each epoch**
            if model.use_cdl and (epoch + 1) % args.get('save_freq', 10) == 0:
                feature_importance = model.get_feature_importance().cpu().numpy()
                os.makedirs(args.get('results_dir', './results'), exist_ok=True)
                np.save(f"{args.get('results_dir', './results')}/feature_importance_epoch_{epoch+1}.npy",
                       feature_importance)

                # Select top features and save indices
                num_top_features = args.get('num_selected_features', 64)
                if model.use_cdl:
                    indices, weights = model.cdl_layer.select_features(num_top_features)
                    np.save(f"{args.get('results_dir', './results')}/selected_features_epoch_{epoch+1}.npy",
                           indices.cpu().numpy())
                    print(f'Selected top {num_top_features} features with weights: '
                          f'mean={torch.mean(weights):.6f}, std={torch.std(weights):.6f}')

        # POST-TRAINING ANALYSIS
        print('Saving reconstructed and original images...')

        # Save model and images
        model.to(device)
        classes_to_save = [0, 1] if hasattr(args, 'classes_to_save') else [0, 1]
        original_images, reconstructed_images = save_reconstructed_and_original_images_cdl(
            model=model,
            traindata=traindata,
            traintargets=traintargets,
            classes=classes_to_save,
            num_images=20,
            device=device,
            save_dir='data/vae-cdl/cdl_vae_reconstructions/'
        )

        if (vggfacesdataset):
            model.cpu()


            if model.use_cdl:
                model.to(device)
                model.eval()

                # Get final feature importance rankings
                with torch.no_grad():
                    final_importance = model.get_feature_importance()

                    # Select different numbers of features for analysis
                    for num_features in [32, 64, 128]:
                        if num_features <= len(final_importance):
                            indices, weights = model.cdl_layer.select_features(num_features)
                            print(f'Top {num_features} features - indices: {indices[:10].cpu().numpy()}, '
                                  f'weights: {weights[:10].cpu().numpy()}')

                            # Save final selections
                            os.makedirs(args.get('results_dir', './results'), exist_ok=True)
                            np.save(f"{args.get('results_dir', './results')}/final_selected_features_{num_features}.npy",
                                   indices.cpu().numpy())
                            np.save(f"{args.get('results_dir', './results')}/final_feature_weights_{num_features}.npy",
                                   weights.cpu().numpy())

                model.cpu()

        torch.save(model.state_dict(), args['vaetrainmodelsavepath'])
        print('Training Variational Autoencoder with CDL done')

        # Plot learning curves
        if 'train_losses' in locals():
            plot_learning_curves(train_losses, val_losses, 'CDL-VAE Learning Curves', args.get('results_dir', './results') + '/cdl_vae_learning_curves.png')

            # Save loss data for later analysis
            np.save(f"{args.get('results_dir', './results')}/train_losses.npy", train_losses)
            np.save(f"{args.get('results_dir', './results')}/val_losses.npy", val_losses)

            print(f"\n=== CDL-VAE Training Complete ===")
            print(f"Final Training Loss: {train_losses['total'][-1]:.6f}")
            print(f"Final Validation Loss: {val_losses['total'][-1]:.6f}")
            print(f"Final CDL Loss: {train_losses['cdl'][-1]:.6f}")

    if(traincnn):
        traincnn_start = time.time()


        if (vggfacesdataset):
            model = models.resnet18(pretrained=True)
            model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
            model.maxpool = nn.Identity()
            optimizer = torch.optim.SGD(
                [
                    {'params': get_parameters(model, bias=False)},
                    {'params': get_parameters(model, bias=True), 'lr': args['lr'] * 2, 'weight_decay': args['weight_decay']},
                ],
                lr=args['lr'],
                momentum=args['momentum'],
                weight_decay=args['weight_decay'])

        device = torch.device("cuda" if args['cuda'] else "cpu")
        model.train()
        model.to(device)

        for epoch in range(1, args['epochs'] + 1):
            train(args, model, train_loader, optimizer, epoch)
        # torch.save(model.state_dict(), args['trainmodelsavepath'])

        model.cpu()
        torch.save(model.state_dict(), args['initialtrainmodelsavepath'])
        print('Training CNN done')
        # sys.exit()
        traincnn_end = time.time()
        print('CNN training time in hours:', (traincnn_end - traincnn_start) / (60 * 60))


    print('Training autoencoder and cnn models completed on mnist database. Start game preprocessing on labelsets.')
    # sys.exit()
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # model = VAE_VGGFACES2(
    #             nc=3,                                    # RGB channels
    #             ndf=32,                                  # Decoder feature dimension
    #             nef=32,                                  # Encoder feature dimension
    #             nz=128,                                  # Latent dimension
    #             device=device,
    #             is_train=True,                           # Training mode
    #             use_cdl=True,                           # Enable CDL
    #             dict_size=args.get('dict_size', 256),   # Dictionary size
    #             cdl_tau=args.get('cdl_tau', 0.1),       # CDL sparsity weight
    #             cdl_p=args.get('cdl_p', 0.5)            # CDL regularization parameter
    #         ).to(device)
    # model.load_state_dict(torch.load(args['vaetrainmodelsavepath']))
    # model = model.to(device)
    # results = compare_reconstructions_by_features(model, test_loader, device)
    if (gamepreprocess):
        classpartitioneddatasets = collections.defaultdict(dict)
        classpartitionedlabels = collections.defaultdict(dict)


        if (vggfacesdataset):
            for argstarget in args['positive_targets']:
                for classlabel in args['negative_target']:
                    classpartitioneddatasets[argstarget][classlabel] = torch.zeros(0, 3, vggface_image_length, vggface_image_length)
                    classpartitionedlabels[argstarget][classlabel] = torch.zeros(0, )

            for argstarget in args['positive_targets']:
                for data, target in train_loader:
                    indexvector1 = target == argstarget
                    for classlabel in args['negative_target']:
                        indexvector2 = target == classlabel

                        # print(target[indexvector1],'target[indexvector1]')
                        classpartitioneddatasets[argstarget][classlabel], classpartitionedlabels[argstarget][classlabel] = \
                            torch.cat((classpartitioneddatasets[argstarget][classlabel], data[indexvector1]), 0), \
                            torch.cat((classpartitionedlabels[argstarget][classlabel], target[indexvector1].float()), 0)

                        classpartitioneddatasets[argstarget][classlabel], classpartitionedlabels[argstarget][classlabel] = \
                            torch.cat((classpartitioneddatasets[argstarget][classlabel], data[indexvector2]), 0), \
                            torch.cat((classpartitionedlabels[argstarget][classlabel], target[indexvector2].float()), 0)


        datasetsfp = open(args['advtraindatasetssavepath'],'wb')
        labelsfp = open(args['advtrainlabelssavepath'],'wb')
        pickle.dump(classpartitioneddatasets,datasetsfp)
        pickle.dump(classpartitionedlabels,labelsfp)
        print('Game Data preprocessing done !')
        # sys.exit()



    if (gamepostprocess):

        advtestdatasets = collections.defaultdict(dict)
        advtestlabels = collections.defaultdict(dict)


        if (vggfacesdataset):
            for argstarget in args['positive_targets']:
                for classlabel in args['negative_target']:
                    advtestdatasets[argstarget][classlabel] = torch.zeros(0, 3, vggface_image_length, vggface_image_length)
                    advtestlabels[argstarget][classlabel] = torch.zeros(0, )


            for argstarget in args['positive_targets']:
                for data, target in test_loader:
                    indexvector1 = target == argstarget
                    for classlabel in args['negative_target']:
                        indexvector2 = target == classlabel

                        advtestdatasets[argstarget][classlabel], advtestlabels[argstarget][classlabel] = \
                            torch.cat((advtestdatasets[argstarget][classlabel], data[indexvector1]), 0), \
                            torch.cat((advtestlabels[argstarget][classlabel], target[indexvector1].float()), 0)

                        advtestdatasets[argstarget][classlabel], advtestlabels[argstarget][classlabel] = \
                            torch.cat((advtestdatasets[argstarget][classlabel], data[indexvector2]), 0), \
                            torch.cat((advtestlabels[argstarget][classlabel], target[indexvector2].float()), 0)



        datasetsfp = open(args['advtestdatasetssavepath'],'wb')
        labelsfp = open(args['advtestlabelssavepath'],'wb')
        pickle.dump(advtestdatasets,datasetsfp)
        pickle.dump(advtestlabels,labelsfp)
        print('Game Data postprocessing done !')
        # sys.exit()


    datasetsfp = open(args['advtestdatasetssavepath'], 'rb')
    labelsfp = open(args['advtestlabelssavepath'], 'rb')
    advtestdatasets = pickle.load(datasetsfp)
    advtestlabels = pickle.load(labelsfp)


    datasetsfp = open(args['advtraindatasetssavepath'], 'rb')
    labelsfp = open(args['advtrainlabelssavepath'], 'rb')
    advtraindatasets = pickle.load(datasetsfp)
    advtrainlabels = pickle.load(labelsfp)

    if (vggfacesdataset):
        autoencodermodel = VAE_VGGFACES2(device=torch.device("cpu"))

    autoencodermodel.load_state_dict(torch.load(args['vaetrainmodelsavepath']))

    if (vggfacesdataset):
        model = models.resnet18(pretrained=True)
        model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        model.maxpool = nn.Identity()

    model.load_state_dict(torch.load(args['initialtrainmodelsavepath']))

    print('Processing labelsets and Loading models done. Start game to generate manipulations.')

    generatemanipulations = True




    if(generatemanipulations):

        game_start = time.time()


        bestwmeans = {}
        bestwstddevs = {}

        weightwmeans = {}
        weightwstddevs = {}


        for argstarget in args['target']:
            for splitlabel in args['negative_target']:
                print('argstarget, splitlabel', argstarget, splitlabel)
                print('=========================================Begin Game======================================================')

                bestiterks = {}
                classkey = str(argstarget) + str(splitlabel)
                if (vggfacesdataset):
                    model = models.resnet18(pretrained=True)
                    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
                    model.maxpool = nn.Identity()

                model.load_state_dict(torch.load(args['initialtrainmodelsavepath']))

                posnegdata = advtraindatasets[argstarget][splitlabel]
                posnegtargets = advtrainlabels[argstarget][splitlabel].float()

                adv_split_loader = torch.utils.data.DataLoader(
                        torch.utils.data.TensorDataset(posnegdata,posnegtargets.long(),posnegdata),
                        batch_size=args['test_batch_size'], shuffle=True)


                error1, overallerror1, tprserrors1 = test(args, model, adv_split_loader)
                print('initial train error1', error1, ' given argstarget, splitlabel ', argstarget, splitlabel)

                testposnegdata = advtestdatasets[argstarget][splitlabel]
                testposnegtargets = advtestlabels[argstarget][splitlabel].float().long()
                test_adv_split_loader = torch.utils.data.DataLoader(
                    torch.utils.data.TensorDataset(testposnegdata, testposnegtargets, testposnegdata),
                    batch_size=args['test_batch_size'], shuffle=True)

                error1, overallerror1, tprserrors1 = test(args, model, test_adv_split_loader)
                print('initial test error1', error1, ' given argstarget, splitlabel ', argstarget, splitlabel)


                copyfile(args['initialtrainmodelsavepath'], args['trainmodelsavepath'])


                posindexvector = posnegtargets == argstarget
                negindexvector = posnegtargets == splitlabel

                encodednegwmean, encodednegwstddev = autoencodermodel.encode(posnegdata[negindexvector])
                encodedposwmean, encodedposwstddev = autoencodermodel.encode(posnegdata[posindexvector])

                encodedposwmean = encodedposwmean.detach()
                encodednegwmean = encodednegwmean.detach()

                encodedposwstddev = encodedposwstddev.detach()
                encodednegwstddev = encodednegwstddev.detach()

                print(encodednegwmean.shape)
                print(encodedposwmean.shape)
                print(posindexvector.shape)


                encodedposnegreconwmean, encodedposnegreconwstddev = autoencodermodel.encode(posnegdata)
                print(encodedposnegreconwmean, encodedposnegreconwstddev,'encodedposnegreconwmean, encodedposnegreconwstddev')
                print('autoencodermodel',autoencodermodel)

                encodedadvrecondata = autoencodermodel.reparameterize(encodedposnegreconwmean, encodedposnegreconwstddev)
                posnegrecondata = autoencodermodel.decode(encodedadvrecondata)
                advrecon_split_loader = torch.utils.data.DataLoader(
                    torch.utils.data.TensorDataset(posnegrecondata, posnegtargets.long(), posnegrecondata),
                    batch_size=args['test_batch_size'], shuffle=True)
                reconerror1, reconoverallerror1, recontprserrors1 = test(args, model, advrecon_split_loader)
                print('after encoding+decoding initial error1', reconerror1, ' given argstarget, splitlabel ', argstarget, splitlabel)


                deltawmean = encodednegwmean.mean(dim=0) - encodedposwmean.mean(dim=0)
                deltawstddev = encodednegwstddev.mean(dim=0) - encodedposwstddev.mean(dim=0)
                deltawmean = deltawmean.view(1,encodednegwmean.shape[1])
                deltawstddev = deltawstddev.view(1,encodednegwstddev.shape[1])



                deltawmean = deltawmean.detach()
                deltawstddev = deltawstddev.detach()

                if (sasearch):


                    weightwmean = encodedposwmean.mean(dim=0)
                    weightwmean = weightwmean.view(1, encodedposwmean.shape[1])

                    weightwstddev = encodedposwstddev.mean(dim=0)
                    weightwstddev = weightwstddev.view(1, encodedposwstddev.shape[1])

                    weightwmean = torch.zeros(weightwmean.shape, dtype=torch.float).clone()
                    weightwstddev = torch.zeros(weightwstddev.shape, dtype=torch.float).clone()

                    weightwmeans[classkey], weightwstddevs[classkey] = weightwmean, weightwstddev
                    bestweightwmean, bestweightwstddev = weightwmean.clone(), weightwstddev.clone()



                else:
                    weightwstddev = Variable(torch.FloatTensor(torch.zeros(1, deltawstddev.size(1))), requires_grad=True)
                    weightwmean = Variable(torch.FloatTensor(torch.zeros(1, deltawmean.size(1))), requires_grad=True)

                playgameflag = True
                gameiter = 0
                alsiter = 1

                while (playgameflag):
                    gameiter += 1
                    model.load_state_dict(torch.load(args['trainmodelsavepath']))
                    alsloopflag = True




                    if (vggfacesdataset):
                        args['error2ssavepath'] = 'data/vae-cdl/vggfaces2manips-error2s-b3' + str(gameiter) + classkey + targetkey + negkey + str(
                            args['numsteps']) + str(args['minpercent']) + str(args['mininc']) + str(args['lambda']) + str(codesize) + '.pkl'
                        args['manipcostssavepath'] = 'data/vae-cdl/vggfaces2manips-manipcosts-b3' + str(gameiter) + classkey + targetkey + negkey + str(
                            args['numsteps']) + str(args['minpercent']) + str(args['mininc']) + str(args['lambda']) + str(codesize) + '.pkl'
                        args['fitnessfnvaluessavepath'] = 'data/vae-cdl/vggfaces2manips-fitnessfnvalues-b3' + str(gameiter) + classkey + targetkey + negkey + str(
                            args['numsteps']) + str(args['minpercent']) + str(args['mininc']) + str(args['lambda']) + str(codesize) + '.pkl'

                        args['weightwmeanssavepath'] = 'data/vae-cdl/vggfaces2manips-weightwmeans-b3' + str(gameiter) + classkey + targetkey + negkey + str(
                            args['numsteps']) + str(args['minpercent']) + str(args['mininc']) + str(args['lambda']) + str(codesize) + '.pkl'
                        args['weightwstddevssavepath'] = 'data/vae-cdl/vggfaces2manips-weightwstddevs-b3' + str(gameiter) + classkey + targetkey + negkey + str(
                            args['numsteps']) + str(args['minpercent']) + str(args['mininc']) + str(args['lambda']) + str(codesize) + '.pkl'



                    initialerror2, initialoverallerror2, initialtprserrors = maskedclassifiererror(posnegdata,
                                                                                                   posnegtargets,
                                                                                                   posindexvector,
                                                                                                   negindexvector,
                                                                                                   args,
                                                                                                   model,
                                                                                                   # deltawmean.detach(),
                                                                                                   # deltawstddev.detach(),
                                                                                                   # weightwmean.detach().numpy()[0],
                                                                                                   # weightwstddev.detach().numpy()[0],
                                                                                                   # encodedposwmean[0:1],
                                                                                                   # encodedposwstddev[0:1],
                                                                                                   encodedposwmean,
                                                                                                   encodedposwstddev,
                                                                                                   deltawmean,
                                                                                                   deltawstddev,
                                                                                                   weightwmean,
                                                                                                   weightwstddev,
                                                                                                   autoencodermodel)
                    if(klnorm):
                        advdata = adversarialmanipulation(posnegdata, posindexvector, negindexvector, deltawmean,
                                                          # encodedposwmean[0:1], encodedposwstddev[0:1],
                                                          encodedposwmean,
                                                          encodedposwstddev,
                                                          deltawstddev,
                                                          weightwmean, weightwstddev, autoencodermodel)

                        initialmanipcost = linesearch_loss_function(posnegdata, advdata, autoencodermodel, args)
                    if (vectornorm):
                        initialmanipcost = torch.norm(weightwmean) + torch.norm(weightwstddev)

                    print('initialmanipcost',initialmanipcost)



                    initialfitnessfnvalue = (initialerror2 - args['lambda'] * initialmanipcost.item())


                    print('weightwmean, weightwstddev', weightwmean, weightwstddev)

                    print('initial fitnessfnvalue', initialfitnessfnvalue)
                    print('initial error2', initialerror2)
                    print('initial manipcost', initialmanipcost)




                    start = time.time()


                    prevpayoff, preverror2, prevmanipcost = initialfitnessfnvalue, initialerror2, initialmanipcost

                    while(alsloopflag):
                        print('Alternating least squares attack started for alsiter:', alsiter)


                        if(alsiter==1):

                            if(attackmean):
                                bestwmeanpayoff, bestwmeanerror2, bestwmeanmanipcost = prevpayoff, preverror2, prevmanipcost
                                print(
                                    '=========================================Begin Mean Attack======================================================')
                                print('bestwmeanpayoff, bestwmeanerror2, bestwmeanmanipcost before mean attack',bestwmeanpayoff, bestwmeanerror2, bestwmeanmanipcost)

                                bestweightwmeancid, bestweightwmean, bestweightwstddev, bestiterks, bestwmeanpayoff, bestwmeanerror2, bestwmeanmanipcost = \
                                    linsearch(bestwmeanpayoff, bestwmeanerror2, bestwmeanmanipcost, args, model,
                                              bestweightwmean, bestweightwstddev, True,
                                          posnegdata, posnegtargets, posindexvector, negindexvector,
                                          deltawmean, deltawstddev, encodedposwmean, encodedposwstddev, autoencodermodel,
                                              bestiterks, classkey, gameiter)
                                print('bestwmeanpayoff, bestwmeanerror2, bestwmeanmanipcost after mean attack',bestwmeanpayoff, bestwmeanerror2, bestwmeanmanipcost)


                                # sys.exit()

                                payoffdiff = (bestwmeanpayoff - prevpayoff)
                                errordiff = (bestwmeanerror2 - preverror2)

                                if(not (attackmean and attackvar)):
                                    if (payoffdiff > 0):
                                        if(controlledthreshold_als):
                                            if (bestwmeanerror2 > args['maxerror_als']):
                                                alsloopflag = False
                                                break
                                            else:
                                                alsloopflag = True
                                        else:
                                            alsloopflag = True
                                    else:
                                        alsloopflag = False



                                print(
                                    '=========================================End Mean Attack======================================================')
                                prevpayoff, preverror2, prevmanipcost = bestwmeanpayoff, bestwmeanerror2, bestwmeanmanipcost
                                print('payoffdiff,alsloopflag,bestweightwmeancid,alsiter',payoffdiff,alsloopflag,bestweightwmeancid,alsiter)


                            if(attackvar):
                                bestwstddevpayoff, bestwstddeverror2, bestwstddevmanipcost = prevpayoff, preverror2, prevmanipcost

                                print(
                                    '=========================================Begin Variance Attack======================================================')

                                print('bestwstddevpayoff, bestwstddeverror2, bestwstddevmanipcost before variance attack',bestwstddevpayoff, bestwstddeverror2, bestwstddevmanipcost)

                                bestweightwstddevcid, bestweightwmean, bestweightwstddev, bestiterks, bestwstddevpayoff, bestwstddeverror2, bestwstddevmanipcost = \
                                    linsearch(bestwstddevpayoff, bestwstddeverror2, bestwstddevmanipcost, args, model,
                                              bestweightwmean, bestweightwstddev, False,
                                          posnegdata, posnegtargets, posindexvector, negindexvector,
                                          deltawmean, deltawstddev, encodedposwmean, encodedposwstddev, autoencodermodel,
                                              bestiterks, classkey, gameiter)


                                print('bestwstddevpayoff, bestwstddeverror2, bestwstddevmanipcost after variance attack',bestwstddevpayoff, bestwstddeverror2, bestwstddevmanipcost)

                                payoffdiff = (bestwstddevpayoff - prevpayoff)
                                errordiff = (bestwstddeverror2 - preverror2)

                                if (payoffdiff > 0):
                                    if(controlledthreshold_als):
                                        if (bestwstddeverror2 > args['maxerror_als']):
                                            alsloopflag = False
                                            break
                                        else:
                                            alsloopflag = True
                                    else:
                                        alsloopflag = True
                                else:
                                    alsloopflag = False

                                print(
                                    '=========================================End Variance Attack======================================================')
                                prevpayoff, preverror2, prevmanipcost = bestwstddevpayoff, bestwstddeverror2, bestwstddevmanipcost
                                print('payoffdiff,alsloopflag,bestweightwstddevcid,alsiter',payoffdiff,alsloopflag,bestweightwstddevcid,alsiter)


                            if(not combinedsinglesattack):
                                if(not combinedattack):
                                    alsiter += 1


                        elif(alsiter>1):
                            if(attackmean):

                                print(
                                    '=========================================Begin Mean Attack======================================================')
                                bestwmeanpayoff, bestwmeanerror2 = prevpayoff, preverror2
                                print('bestwmeanpayoff, bestwmeanerror2 before mean attack',bestwmeanpayoff, bestwmeanerror2)

                                bestweightwmean, bestweightwstddev, bestiterks, bestwmeanpayoff, bestwmeanerror2 \
                                    = linsearchloop(bestweightwmeancid, args, True, deltawmean, deltawstddev, bestweightwmean,
                                              bestweightwstddev, bestwmeanpayoff, bestwmeanerror2, posnegdata,
                                              posnegtargets, posindexvector, negindexvector, model, encodedposwmean,
                                              encodedposwstddev, autoencodermodel, bestiterks, classkey, gameiter)

                                print('bestwmeanpayoff, bestwmeanerror2 after mean attack',bestwmeanpayoff, bestwmeanerror2)


                                payoffdiff = (bestwmeanpayoff - prevpayoff)
                                errordiff = (bestwmeanerror2 - preverror2)

                                if(not (attackmean and attackvar)):
                                    if (payoffdiff > 0):
                                        if(controlledthreshold_als):
                                            if (bestwmeanerror2 > args['maxerror_als']):
                                                alsloopflag = False
                                                break
                                            else:
                                                alsloopflag = True
                                        else:
                                            alsloopflag = True
                                    else:
                                        alsloopflag = False

                                print(
                                    '=========================================End Mean Attack======================================================')
                                prevpayoff, preverror2 = bestwmeanpayoff, bestwmeanerror2
                                print('payoffdiff,alsloopflag,bestweightwmeancid,alsiter',payoffdiff,alsloopflag,bestweightwmeancid,alsiter)

                            if (attackvar):
                            # if(not combinedattack):
                                print(
                                    '=========================================Begin Variance Attack======================================================')
                                bestwstddevpayoff, bestwstddeverror2 = prevpayoff, preverror2

                                print('bestwstddevpayoff, bestwstddeverror2 before variance attack', bestwstddevpayoff,
                                      bestwstddeverror2)

                                bestweightwmean, bestweightwstddev, bestiterks, bestwstddevpayoff, bestwstddeverror2 \
                                        = linsearchloop(bestweightwstddevcid, args, False, deltawmean, deltawstddev, bestweightwmean,
                                                  bestweightwstddev, bestwstddevpayoff, bestwstddeverror2, posnegdata,
                                                  posnegtargets, posindexvector, negindexvector, model, encodedposwmean,
                                                  encodedposwstddev, autoencodermodel, bestiterks, classkey, gameiter)

                                print('bestwstddevpayoff, bestwstddeverror2 after variance attack',bestwstddevpayoff, bestwstddeverror2)


                                payoffdiff = (bestwstddevpayoff - prevpayoff)
                                errordiff = (bestwstddeverror2 - preverror2)
                                if (payoffdiff > 0):
                                    if(controlledthreshold_als):
                                        if (bestwstddeverror2 > args['maxerror_als']):
                                            alsloopflag = False
                                            break
                                        else:
                                            alsloopflag = True
                                    else:
                                        alsloopflag = True
                                else:
                                    alsloopflag = False


                                print(
                                    '=========================================End Variance Attack======================================================')
                                prevpayoff, preverror2 = bestwstddevpayoff, bestwstddeverror2
                                print('payoffdiff,alsloopflag,bestweightwstddevcid,alsiter', payoffdiff, alsloopflag, bestweightwstddevcid, alsiter)

                            alsiter += 1


                        print('Alternating least squares attack completed for alsiter:',alsiter)

                    end = time.time()
                    print('Alternating least squares search time in hours:',(end - start) / (60*60))

                    print('bestweightwmean',bestweightwmean)
                    print('bestweightwstddev',bestweightwstddev)
                    # sys.exit()

                    besterror2, bestoverallerror2, besttprserrors = maskedclassifiererror(posnegdata,
                                                                                         posnegtargets,
                                                                                         posindexvector,
                                                                                         negindexvector,
                                                                                         args,
                                                                                         model,
                                                                                         # deltawmean.detach(),
                                                                                         # deltawstddev.detach(),
                                                                                         # weightwmean.detach().numpy()[0],
                                                                                         # weightwstddev.detach().numpy()[0],
                                                                                         #  encodedposwmean[0:1],
                                                                                         #  encodedposwstddev[0:1],
                                                                                          encodedposwmean,
                                                                                          encodedposwstddev,
                                                                                          deltawmean,
                                                                                          deltawstddev,
                                                                                          bestweightwmean,
                                                                                          bestweightwstddev,
                                                                                          autoencodermodel)
                    if(klnorm):
                        advdata = adversarialmanipulation(posnegdata, posindexvector, negindexvector, deltawmean,
                                                          # encodedposwmean[0:1], encodedposwstddev[0:1],
                                                          encodedposwmean,
                                                          encodedposwstddev,
                                                          deltawstddev,
                                                          bestweightwmean, bestweightwstddev, autoencodermodel)
                        bestmanipcost = linesearch_loss_function(posnegdata, advdata, autoencodermodel, args)

                    if(vectornorm):
                        bestmanipcost = torch.norm(bestweightwmean) + torch.norm(bestweightwstddev)

                    # bestmanipcost = bestmanipcost.detatch()
                    bestfitnessfnvalue = (besterror2 - args['lambda'] * bestmanipcost.item())

                    gamefitnessfndiff = (bestfitnessfnvalue - initialfitnessfnvalue)
                    gamepercentfitnessfndiff = ((bestfitnessfnvalue - initialfitnessfnvalue) / initialfitnessfnvalue) * 100

                    gameerrordiff = besterror2 - initialerror2

                    print('besterror2', besterror2)
                    print('bestmanipcost', bestmanipcost)
                    print('bestfitnessfnvalue', bestfitnessfnvalue)

                    if (gamefitnessfndiff > 0):

                        print('found a valid manipulation across game iters')
                        # sys.exit()

                        if (controlledthreshold_game):
                            if(besterror2 > args['maxerror_game']):
                                playgameflag = False
                                break
                            else:
                                playgameflag = True
                        else:
                            playgameflag = True


                        maskedposnegdata = adversarialmanipulation(posnegdata, posindexvector, negindexvector, deltawmean,
                                                                   # encodedposwmean[0:1], encodedposwstddev[0:1],
                                                                   encodedposwmean,
                                                                   encodedposwstddev,
                                                                   deltawstddev,
                                                                   bestweightwmean, bestweightwstddev, autoencodermodel)

                        advdata = maskedposnegdata
                        advtargets = posnegtargets
                        advdata, advtargets = torch.cat((advdata, traindata), 0), torch.cat((advtargets, traintargets), 0)
                        adv_train_loader = torch.utils.data.DataLoader(
                            torch.utils.data.TensorDataset(advdata, advtargets.long()),
                            batch_size=args['test_batch_size'], shuffle=True)

                        print('advdata.shape',advdata.shape)
                        print('advtargets.shape',advtargets.shape)


                        if (vggfacesdataset):
                            model = models.resnet18(pretrained=True)
                            model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
                            model.maxpool = nn.Identity()
                            optimizer = torch.optim.SGD(
                            [
                                {'params': get_parameters(model, bias=False)},
                                {'params': get_parameters(model, bias=True), 'lr': args['lr'] * 2,
                                 'weight_decay': args['weight_decay']},
                            ],
                            lr=args['lr'],
                            momentum=args['momentum'],
                            weight_decay=args['weight_decay'])

                        device = torch.device("cuda" if args['cuda'] else "cpu")
                        model.train()
                        model.to(device)

                        # model.load_state_dict(torch.load(args['initialtrainmodelsavepath']))
                        for epoch in range(1, args['epochs'] + 1):
                            train(args, model, adv_train_loader, optimizer, epoch)

                        model.cpu()
                        torch.save(model.state_dict(), args['trainmodelsavepath'])

                    else:
                        playgameflag = False

                    # initialfitnessfnvalue = bestfitnessfnvalue
                    weightwmean, weightwstddev = bestweightwmean.clone(), bestweightwstddev.clone()

                    print('splitlabel', splitlabel)
                    print('bestiterks', bestiterks)
                    print('gamefitnessfndiff', gamefitnessfndiff)
                    print('gamefitnessfndiff>0', gamefitnessfndiff>0)

                    # sys.exit()

                bestwmeans[classkey] = bestweightwmean
                bestwstddevs[classkey] = bestweightwstddev

                print(classkey, bestwmeans, bestwstddevs)
                # print(maskedclassifiererror(posnegdata, posnegtargets, args, model, bestalphas[k], autoencodermodel))
                print('argstarget, splitlabel', argstarget, splitlabel)
                print('gameiter', gameiter)
                print('===============================================End Game================================================')
                # sys.exit()

        bestwmeansfp = open(args['bestwmeanssavepath'], 'wb')
        pickle.dump(bestwmeans, bestwmeansfp)

        bestwstddevsfp = open(args['bestwstddevssavepath'], 'wb')
        pickle.dump(bestwstddevs, bestwstddevsfp)
        print('Saved best adversarial manipulations to disk')
        # sys.exit()
        # Add this code after the game loop completes (after the pickle.dump operations)

        print("Generating and saving final manipulated images...")

        # Create directories for final manipulated images
        os.makedirs('data/vae-cdl/final_manipulations/', exist_ok=True)
        os.makedirs('data/vae-cdl/final_originals/', exist_ok=True)

        # Generate final manipulated images for each class combination
        for argstarget in args['target']:
            for splitlabel in args['negative_target']:
                classkey = str(argstarget) + str(splitlabel)

                if classkey in bestwmeans and classkey in bestwstddevs:
                    print(f"Generating final images for target {argstarget} vs {splitlabel}...")

                    # Get the original data
                    posnegdata = advtraindatasets[argstarget][splitlabel]
                    posnegtargets = advtrainlabels[argstarget][splitlabel].float()

                    # Create index vectors
                    posindexvector = posnegtargets == argstarget
                    negindexvector = posnegtargets == splitlabel

                    # Encode the data
                    encodednegwmean, encodednegwstddev = autoencodermodel.encode(posnegdata[negindexvector])
                    encodedposwmean, encodedposwstddev = autoencodermodel.encode(posnegdata[posindexvector])

                    # Get delta values
                    deltawmean = encodednegwmean.mean(dim=0) - encodedposwmean.mean(dim=0)
                    deltawstddev = encodednegwstddev.mean(dim=0) - encodedposwstddev.mean(dim=0)
                    deltawmean = deltawmean.view(1, encodednegwmean.shape[1])
                    deltawstddev = deltawstddev.view(1, encodednegwstddev.shape[1])

                    # Get best weights
                    bestweightwmean = bestwmeans[classkey]
                    bestweightwstddev = bestwstddevs[classkey]

                    # Generate final manipulated images
                    final_advdata = adversarialmanipulation(posnegdata, posindexvector, negindexvector,
                                                           deltawmean, encodedposwmean, encodedposwstddev,
                                                           deltawstddev, bestweightwmean, bestweightwstddev,
                                                           autoencodermodel)

                    # Save a subset of manipulated images (first 10 from each class)
                    num_images_to_save = min(10, posnegdata.shape[0])

                    for i in range(num_images_to_save):
                        # Original image
                        orig_img = posnegdata[i]
                        orig_class = posnegtargets[i].item()

                        # Manipulated image
                        manip_img = final_advdata[i]

                        # Resize and save original
                        orig_img_resized = transforms.Compose([
                            transforms.ToPILImage(),
                            transforms.Resize(256),
                            transforms.ToTensor()
                        ])(orig_img)

                        # Resize and save manipulated
                        manip_img_resized = transforms.Compose([
                            transforms.ToPILImage(),
                            transforms.Resize(256),
                            transforms.ToTensor()
                        ])(manip_img)

                        # Save images with descriptive names
                        orig_filename = f'data/vae-cdl/final_originals/orig_{argstarget}v{splitlabel}_{i}_class{orig_class}.jpeg'
                        manip_filename = f'data/vae-cdl/final_manipulations/manip_{argstarget}v{splitlabel}_{i}_class{orig_class}.jpeg'

                        torchvision.utils.save_image(orig_img_resized, orig_filename)
                        torchvision.utils.save_image(manip_img_resized, manip_filename)

                    print(f"Saved {num_images_to_save} image pairs for {argstarget} vs {splitlabel}")

        print("Final manipulated images saved successfully!")

        game_end = time.time()
        print('Game optimization time in hours:', (game_end - game_start) / (60 * 60))



    print("=== CALCULATING FINAL ERROR2 IMMEDIATELY AFTER OPTIMIZATION ===")

    # Calculate for each target-class combination
    argstarget = list(args['target'])[0]
    classlabel = list(args['negative_target'])[0]
    classkey = str(argstarget) + str(classlabel)

    # Load original model
    if vggfacesdataset:
        orig_model = models.resnet18(pretrained=True)
        orig_model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        orig_model.maxpool = nn.Identity()

    orig_model.load_state_dict(torch.load(args['initialtrainmodelsavepath']))

    # Get test data
    test_data = advtestdatasets[argstarget][classlabel]
    test_targets = advtestlabels[argstarget][classlabel].float()

    # Generate adversarial test data using best parameters
    pos_idx = test_targets == argstarget
    neg_idx = test_targets == classlabel

    encoded_neg_mean, encoded_neg_std = autoencodermodel.encode(test_data[neg_idx])
    encoded_pos_mean, encoded_pos_std = autoencodermodel.encode(test_data[pos_idx])

    delta_mean = encoded_neg_mean.mean(dim=0) - encoded_pos_mean.mean(dim=0)
    delta_std = encoded_neg_std.mean(dim=0) - encoded_pos_std.mean(dim=0)
    delta_mean = delta_mean.view(1, encoded_neg_mean.shape[1])
    delta_std = delta_std.view(1, encoded_neg_std.shape[1])

    # Generate adversarial data
    adv_test_data = adversarialmanipulation(
        test_data, pos_idx, neg_idx, delta_mean,
        encoded_pos_mean, encoded_pos_std, delta_std,
        bestwmeans[classkey], bestwstddevs[classkey], autoencodermodel
    )

    # Calculate errors
    clean_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(test_data, test_targets.long(), test_data),
        batch_size=args['test_batch_size'], shuffle=True)

    adv_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(adv_test_data, test_targets.long(), adv_test_data),
        batch_size=args['test_batch_size'], shuffle=True)

    final_error1, _, _ = test(args, orig_model, clean_loader)
    final_error2, _, _ = test(args, orig_model, adv_loader)

    print("=== FINAL ERRORS (CALCULATED EARLY) ===")
    print(f'Final Error1 (Clean): {final_error1}')
    print(f'Final Error2 (Adversarial): {final_error2}')
    print(f'Attack Effectiveness: {final_error2 - final_error1:.4f}')

    bestwmeansfp = open(args['bestwmeanssavepath'], 'rb')
    bestwstddevsfp = open(args['bestwstddevssavepath'], 'rb')

    bestwmeans = pickle.load(bestwmeansfp)
    bestwstddevs = pickle.load(bestwstddevsfp)

    print('starwmeans',bestwmeans)
    print('starwstddevs',bestwstddevs)


    datasetsfp = open(args['advtestdatasetssavepath'], 'rb')
    labelsfp = open(args['advtestlabelssavepath'], 'rb')
    advtestdatasets = pickle.load(datasetsfp)
    advtestlabels = pickle.load(labelsfp)



    if (vggfacesdataset):

        finaladvtraindata = torch.zeros(0, 3, 64, 64)
        finaladvtestdata = torch.zeros(0, 3, 64, 64)
        originaladvtestdata = torch.zeros(0, 3, 64, 64)
        originaladvtraindata = torch.zeros(0, 3, 64, 64)

    finaladvtraintargets = torch.zeros(0, )
    finaladvtesttargets = torch.zeros(0, )
    originaladvtesttargets = torch.zeros(0, )
    originaladvtraintargets = torch.zeros(0, )


    argstarget=list(args['target'])[0]
    classlabel=list(args['negative_target'])[0]

    classkey = str(argstarget) + str(classlabel)
    bestweightwmean = bestwmeans[classkey]
    bestweightwstddev = bestwstddevs[classkey]

    if(benchmarkalphas==False):
        posnegtraindata = advtraindatasets[argstarget][classlabel]
        posnegtraintargets = advtrainlabels[argstarget][classlabel].float()

        posnegtestdata = advtestdatasets[argstarget][classlabel]
        posnegtesttargets = advtestlabels[argstarget][classlabel].float()

        finaladvtraindata = torch.cat((posnegtraindata, finaladvtraindata), 0)
        finaladvtraintargets = torch.cat((posnegtraintargets, finaladvtraintargets), 0)

        postraindataindexvector = posnegtraintargets == argstarget
        negtraindataindexvector = posnegtraintargets == classlabel
        encodednegtraindatawmean, encodednegtraindatawstddev = autoencodermodel.encode(posnegtraindata[negtraindataindexvector])
        encodedpostraindatawmean, encodedpostraindatawstddev = autoencodermodel.encode(posnegtraindata[postraindataindexvector])
        encodedpostraindatawmean = encodedpostraindatawmean.detach()
        encodednegtraindatawmean = encodednegtraindatawmean.detach()
        encodedpostraindatawstddev = encodedpostraindatawstddev.detach()
        encodednegtraindatawstddev = encodednegtraindatawstddev.detach()
        deltawmean_traindata = encodednegtraindatawmean.mean(dim=0) - encodedpostraindatawmean.mean(dim=0)
        deltawstddev_traindata = encodednegtraindatawstddev.mean(dim=0) - encodedpostraindatawstddev.mean(dim=0)
        deltawmean_traindata = deltawmean_traindata.view(1, encodednegtraindatawmean.shape[1])
        deltawstddev_traindata = deltawstddev_traindata.view(1, encodednegtraindatawstddev.shape[1])
        maskedposnegtraindata = adversarialmanipulation(posnegtraindata, postraindataindexvector, negtraindataindexvector, deltawmean_traindata,
                                                   # encodedposwmean[0:1], encodedposwstddev[0:1],
                                                   encodedpostraindatawmean,
                                                   encodedpostraindatawstddev,
                                                   deltawstddev_traindata,
                                                   bestweightwmean, bestweightwstddev, autoencodermodel)
        maskedposnegtraindata = maskedposnegtraindata.detach()


        if (not savemanipulations):
            if (mnistdataset):
                model = Net()
            if (vggfacesdataset):
                model = models.resnet18(pretrained=True)
                model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
                model.maxpool = nn.Identity()

            model.load_state_dict(torch.load(args['initialtrainmodelsavepath']))

            besterror2, bestoverallerror2, besttprserrors = maskedclassifiererror(posnegtraindata,
                                                                                  posnegtraintargets,
                                                                                  postraindataindexvector,
                                                                                  negtraindataindexvector,
                                                                                  args,
                                                                                  model,
                                                                                  # deltawmean.detach(),
                                                                                  # deltawstddev.detach(),
                                                                                  # weightwmean.detach().numpy()[0],
                                                                                  # weightwstddev.detach().numpy()[0],
                                                                                  #  encodedposwmean[0:1],
                                                                                  #  encodedposwstddev[0:1],
                                                                                  encodedpostraindatawmean,
                                                                                  encodedpostraindatawstddev,
                                                                                  deltawmean_traindata,
                                                                                  deltawstddev_traindata,
                                                                                  bestweightwmean,
                                                                                  bestweightwstddev,
                                                                                  autoencodermodel)
            print('besterror2 train after',besterror2)
            print('bestoverallerror2 train after', bestoverallerror2)
    # sys.exit()

    postestdataindexvector = posnegtesttargets == argstarget
    negtestdataindexvector = posnegtesttargets == classlabel
    encodednegtestdatawmean, encodednegtestdatawstddev = autoencodermodel.encode(posnegtestdata[negtestdataindexvector])
    encodedpostestdatawmean, encodedpostestdatawstddev = autoencodermodel.encode(posnegtestdata[postestdataindexvector])
    encodedpostestdatawmean = encodedpostestdatawmean.detach()
    encodednegtestdatawmean = encodednegtestdatawmean.detach()
    encodedpostestdatawstddev = encodedpostestdatawstddev.detach()
    encodednegtestdatawstddev = encodednegtestdatawstddev.detach()
    deltawmean_testdata = encodednegtestdatawmean.mean(dim=0) - encodedpostestdatawmean.mean(dim=0)
    deltawstddev_testdata = encodednegtestdatawstddev.mean(dim=0) - encodedpostestdatawstddev.mean(dim=0)
    deltawmean_testdata = deltawmean_testdata.view(1, encodednegtestdatawmean.shape[1])
    deltawstddev_testdata = deltawstddev_testdata.view(1, encodednegtestdatawstddev.shape[1])
    maskedposnegtestdata = adversarialmanipulation(posnegtestdata, postestdataindexvector, negtestdataindexvector, deltawmean_testdata,
                                               # encodedposwmean[0:1], encodedposwstddev[0:1],
                                               encodedpostestdatawmean,
                                               encodedpostestdatawstddev,
                                               deltawstddev_testdata,
                                               bestweightwmean, bestweightwstddev, autoencodermodel)

    if(benchmarkalphas==False):
        print('posnegtestdata.shape',posnegtestdata.shape)
        print('deltawmean_traindata.shape',deltawmean_traindata.shape)
        print('deltawstddev_traindata.shape',deltawstddev_traindata.shape)
        print('encodedpostraindatawmean.shape',encodedpostraindatawmean.shape)
        print('encodedpostraindatawstddev.shape',encodedpostraindatawstddev.shape)


    maskedposnegtestdata = maskedposnegtestdata.detach()


    besterror2, bestoverallerror2, besttprserrors = maskedclassifiererror(posnegtestdata,
                                                                          posnegtesttargets,
                                                                          postestdataindexvector,
                                                                          negtestdataindexvector,
                                                                          args,
                                                                          model,
                                                                          encodedpostestdatawmean,
                                                                          encodedpostestdatawstddev,
                                                                          deltawmean_testdata,
                                                                          deltawstddev_testdata,
                                                                          bestweightwmean,
                                                                          bestweightwstddev,
                                                                          autoencodermodel)


    if (not savemanipulations):
        if (mnistdataset):
            model = Net()
        if (vggfacesdataset):
            model = models.resnet18(pretrained=True)
            model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
            model.maxpool = nn.Identity()
        model.load_state_dict(torch.load(args['initialtrainmodelsavepath']))

        initial_test_data_loader = torch.utils.data.DataLoader(
            torch.utils.data.TensorDataset(posnegtestdata, posnegtesttargets.long(), posnegtestdata),
            batch_size=args['test_batch_size'], shuffle=True, num_workers=4)

        final_error1, final_overallerror1, final_tprserrors1 = test(args, model,
                                                                    initial_test_data_loader)  # Train on x-train and test on x-test
        print('final error1', final_error1)
        print('final overall error1', final_overallerror1)

    print('besterror2 test after', besterror2)
    print('bestoverallerror2 test after', bestoverallerror2)



    originaladvtraindata = torch.cat((posnegtraindata, originaladvtraindata), 0)
    originaladvtraintargets = torch.cat((posnegtraintargets, originaladvtraintargets), 0)
    finaladvtraindata = torch.cat((maskedposnegtraindata, finaladvtraindata), 0)
    finaladvtraintargets = torch.cat((posnegtraintargets, finaladvtraintargets), 0)

    if(benchmarkalphas==True):
        finaladvtraindata = torch.cat((originaladvtraindata, finaladvtraindata), 0)
        finaladvtraintargets = torch.cat((originaladvtraintargets, finaladvtraintargets), 0)
    final_adv_train_data_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(finaladvtraindata, finaladvtraintargets.long()),
        batch_size=args['test_batch_size'], shuffle=True, num_workers=4)


    if (vggfacesdataset):
        args['epochs'] = 10
        model = models.resnet18(pretrained=True)
        model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        model.maxpool = nn.Identity()
        optimizer = torch.optim.SGD(
            [
                {'params': get_parameters(model, bias=False)},
                {'params': get_parameters(model, bias=True), 'lr': args['lr'] * 2,
                 'weight_decay': args['weight_decay']},
            ],
            lr=args['lr'],
            momentum=args['momentum'],
            weight_decay=args['weight_decay'])

    device = torch.device("cuda" if args['cuda'] else "cpu")
    model.train()
    model.to(device)

    for epoch in range(1, args['epochs'] + 1):
        train(args, model, final_adv_train_data_loader, optimizer, epoch)

    model.cpu()

    originaladvtestdata = torch.cat((posnegtestdata, originaladvtestdata), 0)
    originaladvtesttargets = torch.cat((posnegtesttargets, originaladvtesttargets), 0)
    finaladvtestdata = torch.cat((maskedposnegtestdata, finaladvtestdata), 0)
    finaladvtesttargets = torch.cat((posnegtesttargets, finaladvtesttargets), 0)

    if(benchmarkalphas==True):
        finaladvtestdata = torch.cat((originaladvtestdata, finaladvtestdata), 0)
        finaladvtesttargets = torch.cat((originaladvtesttargets, finaladvtesttargets), 0)
    final_adv_test_data_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(finaladvtestdata, finaladvtesttargets.long(), finaladvtestdata),
        batch_size=args['test_batch_size'], shuffle=True, num_workers=4)

    final_error3_1, final_overallerror3_1, final_tprserrors3_1 = test(args, model, final_adv_test_data_loader)
    print('final error3_1', final_error3_1)
    print('final overallerror3_1', final_overallerror3_1)
    finaladvtraintargets = torch.zeros(0, )
    finaladvtesttargets = torch.zeros(0, )
    originaladvtesttargets = torch.zeros(0, )
    originaladvtraintargets = torch.zeros(0, )

    print('bestwmeanssavepath', args['bestwmeanssavepath'])
    args['positive_targets'] = [4]
    args['negative_target'] = [10]


    for argstarget in args['positive_targets']:
        for classlabel in args['negative_target']:

            posnegtraindata = advtraindatasets[argstarget][classlabel]
            posnegtraintargets = advtrainlabels[argstarget][classlabel].float()

            finaladvtraindata = torch.cat((posnegtraindata, finaladvtraindata), 0)
            finaladvtraintargets = torch.cat((posnegtraintargets, finaladvtraintargets), 0)

            classkey = str(argstarget) + str(classlabel)

            # TO DO : Update with runs params for each target
            if(argstarget==2):
                # codesize = 50
                args['numsteps'] = 7
                args['lambda'] = 0.8
                args['target']=set([2])
            elif(argstarget==3):
                # codesize = 50
                args['numsteps'] = 12
                args['lambda'] = 0.8
                args['target']=set([3])
            elif (argstarget == 5):
                # codesize = 50
                args['numsteps'] = 10
                args['lambda'] = 7.0
                args['target']=set([5])
            elif (argstarget == 6):
                # codesize = 50
                args['numsteps'] = 15
                args['lambda'] = 0.8
                args['target']=set([6])
            elif (argstarget == 9):
                # codesize = 50
                args['numsteps'] = 10
                args['lambda'] = 0.8
                args['target']=set([9])

            targetkey = ''.join(map(str, args['target']))

            print('bestwmeanssavepath',args['bestwmeanssavepath'])

            bestwmeansfp = open(args['bestwmeanssavepath'], 'rb')
            bestwstddevsfp = open(args['bestwstddevssavepath'], 'rb')
            bestwmeans = pickle.load(bestwmeansfp)
            bestwstddevs = pickle.load(bestwstddevsfp)

            bestweightwmean = bestwmeans[classkey]
            bestweightwstddev = bestwstddevs[classkey]

            postraindataindexvector = posnegtraintargets == argstarget
            negtraindataindexvector = posnegtraintargets == classlabel
            encodednegtraindatawmean, encodednegtraindatawstddev = autoencodermodel.encode(posnegtraindata[negtraindataindexvector])
            encodedpostraindatawmean, encodedpostraindatawstddev = autoencodermodel.encode(posnegtraindata[postraindataindexvector])
            encodedpostraindatawmean = encodedpostraindatawmean.detach()
            encodednegtraindatawmean = encodednegtraindatawmean.detach()
            encodedpostraindatawstddev = encodedpostraindatawstddev.detach()
            encodednegtraindatawstddev = encodednegtraindatawstddev.detach()
            deltawmean_traindata = encodednegtraindatawmean.mean(dim=0) - encodedpostraindatawmean.mean(dim=0)
            deltawstddev_traindata = encodednegtraindatawstddev.mean(dim=0) - encodedpostraindatawstddev.mean(dim=0)
            deltawmean_traindata = deltawmean_traindata.view(1, encodednegtraindatawmean.shape[1])
            deltawstddev_traindata = deltawstddev_traindata.view(1, encodednegtraindatawstddev.shape[1])
            maskedposnegtraindata = adversarialmanipulation(posnegtraindata, postraindataindexvector, negtraindataindexvector, deltawmean_traindata,
                                                       # encodedposwmean[0:1], encodedposwstddev[0:1],
                                                       encodedpostraindatawmean,
                                                       encodedpostraindatawstddev,
                                                       deltawstddev_traindata,
                                                       bestweightwmean, bestweightwstddev, autoencodermodel)
            maskedposnegtraindata = maskedposnegtraindata.detach()


            originaladvtraindata = torch.cat((posnegtraindata, originaladvtraindata), 0)
            originaladvtraintargets = torch.cat((posnegtraintargets, originaladvtraintargets), 0)
            finaladvtraindata = torch.cat((maskedposnegtraindata, finaladvtraindata), 0)
            finaladvtraintargets = torch.cat((posnegtraintargets, finaladvtraintargets), 0)


    finaladvtraindata = torch.cat((originaladvtraindata, finaladvtraindata), 0)
    finaladvtraintargets = torch.cat((originaladvtraintargets, finaladvtraintargets), 0)
    final_adv_train_data_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(finaladvtraindata, finaladvtraintargets.long()),
        batch_size=args['test_batch_size'], shuffle=True, num_workers=4)

    optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])
    for epoch in range(1, args['epochs'] + 1):
        # train(args, model, final_adv_train_data_loader, optimizer, 1)
        train(args, model, final_adv_train_data_loader, optimizer, epoch)
    torch.save(model.state_dict(), args['finaltrainmodelsavepath'])
    model.load_state_dict(torch.load(args['finaltrainmodelsavepath']))

    final_error3_1, final_overallerror3_1, final_tprserrors3_1 = test(args, model, final_adv_test_data_loader)
    print('final error3_1', final_error3_1)
    print('final overallerror3_1', final_overallerror3_1)

    # sys.exit()



    # Clear below after saving manipulated and misclassified images for all input datasets

    if(not savemanipulations):
        finaladvtestdata = torch.cat((originaladvtestdata, finaladvtestdata), 0)
        finaladvtesttargets = torch.cat((originaladvtesttargets, finaladvtesttargets), 0)
    # Combining original data with manipulated data because testing error is too high on manipulated data alone.
        # To manually find manipulated data remove original data
    print('finaladvtestdata.shape',finaladvtestdata.shape)
    print('originaladvtestdata.shape',originaladvtestdata.shape)

    final_adv_test_data_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(finaladvtestdata, finaladvtesttargets.long(), finaladvtestdata),
        batch_size=args['test_batch_size'], shuffle=True, num_workers=4)
    if(savemanipulations):
        final_adv_test_data_loader = torch.utils.data.DataLoader(
            torch.utils.data.TensorDataset(finaladvtestdata, finaladvtesttargets.long(), originaladvtestdata),
            batch_size=args['test_batch_size'], shuffle=True, num_workers=4)

    initial_test_data_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(originaladvtestdata, originaladvtesttargets.long(), originaladvtestdata),
        batch_size=args['test_batch_size'], shuffle=True, num_workers=4)

    model = Net()
    model.load_state_dict(torch.load(args['initialtrainmodelsavepath']))

    if(not savemanipulations):
        final_error1, final_overallerror1, final_tprserrors1 = test(args, model,
                                                                    initial_test_data_loader)  # Train on x-train and test on x-test
        print('final error1', final_error1)
        print('final overall error1', final_overallerror1)

    final_error2_1, final_overallerror2_1, final_tprserrors2_1 = test(args, model, final_adv_test_data_loader)


    finaladvtraindata = torch.cat((originaladvtraindata, finaladvtraindata), 0)
    finaladvtraintargets = torch.cat((originaladvtraintargets, finaladvtraintargets), 0)
    final_adv_train_data_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(finaladvtraindata, finaladvtraintargets.long()),
        batch_size=args['test_batch_size'], shuffle=True, num_workers=4)

    optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])
    for epoch in range(1, args['epochs'] + 1):
    # train(args, model, final_adv_train_data_loader, optimizer, 1)
        train(args, model, final_adv_train_data_loader, optimizer, epoch)
    torch.save(model.state_dict(), args['finaltrainmodelsavepath'])

    model.load_state_dict(torch.load(args['finaltrainmodelsavepath']))

    final_error3_1, final_overallerror3_1, final_tprserrors3_1 = test(args, model, final_adv_test_data_loader)
    print('final error3_1', final_error3_1)
    print('final overallerror3_1', final_overallerror3_1)

    sys.stdout = output_capture.original_stdout
    output_capture.close()
    print(f"All output saved to: {log_file_path}")

In [ ]:
playgame()

Data preprocessing done !
data.shape target.shape torch.Size([879, 3, 64, 64]) torch.Size([879])
traindata.shape traintargets.shape torch.Size([0, 3, 64, 64]) torch.Size([0])
data.shape target.shape torch.Size([890, 3, 64, 64]) torch.Size([890])
traindata.shape traintargets.shape torch.Size([879, 3, 64, 64]) torch.Size([879])
Training autoencoder and cnn models completed on mnist database. Start game preprocessing on labelsets.
Game Data preprocessing done !
Game Data postprocessing done !


C:\Users\College\miniconda3\envs\tfenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\College\miniconda3\envs\tfenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Processing labelsets and Loading models done. Start game to generate manipulations.
argstarget, splitlabel 0 1
=========================================Begin Game======================================================
batcherror=====================,savemanipulations 3.125 True
batcherror=====================,savemanipulations 3.076923076923077 True
batcherror=====================,savemanipulations 4.255319148936165 True
batcherror=====================,savemanipulations 4.0000000000000036 True
batcherror=====================,savemanipulations 3.7735849056603765 True
batcherror=====================,savemanipulations 4.6875 True
batcherror=====================,savemanipulations 3.9301310043668103 True
batcherror=====================,savemanipulations 3.435114503816794 True
batcherror=====================,savemanipulations 3.0100334448160515 True
batcherror=====================,savemanipulations 2.6548672566371723 True
batcherror=====================,savemanipulations 2.4390243902439046 Tr


KeyboardInterrupt



In [ ]:
# Load and check the actual selections
features_32 = np.load('data/vae-cdl/vae-cdl_results/final_selected_features_32.npy')
features_64 = np.load('data/vae-cdl/vae-cdl_results/final_selected_features_64.npy')
features_128 = np.load('data/vae-cdl/vae-cdl_results/final_selected_features_128.npy')

print(f"Actual 32 features selected: {len(features_32)}")
print(f"Actual 64 features selected: {len(features_64)}")
print(f"Actual 128 features selected: {len(features_128)}")
print(features_128)
weights_128 = np.load('data/vae-cdl/vae-cdl_results/final_feature_weights_128.npy')
features_128 = np.load('data/vae-cdl/vae-cdl_results/final_selected_features_128.npy')
print(weights_128)

Actual 32 features selected: 32
Actual 64 features selected: 64
Actual 128 features selected: 128
[ 23  57  37   8  47  91  27  56 122 103  40  70  54  58  65  80   7  77
  63  60  86  15  83  87  39 120  84   1  29  25 118   5  18  22  72 115
  66  61  67  51  31  48 110  12  93  16  62  41  30 101  94 126  49  73
 125   2  90  17 105  75   4  32  50 108 116 123 109  45   9  85  10  36
 121  43 102  78  11 106  21  26  81   0  34  88  59 111 127  44  68   3
  53  28  35  92  79 104 117  33  71 100  74  97 113  98  55 112  52  13
   6  19  38 124  76  42  24  95 114  89 119  14  69  20  46  96  64  99
  82 107]
[0.60053474 0.56347847 0.5537612  0.49599534 0.48453644 0.4728507
 0.45559973 0.32233465 0.2898386  0.2700378  0.16890667 0.13685752
 0.13374539 0.1154585  0.10297903 0.07351716 0.05367621 0.01013344
 0.00857582 0.00685169 0.00676725 0.00676474 0.00667672 0.00641071
 0.00639319 0.00638682 0.00612646 0.00588784 0.00586006 0.00576418
 0.00569854 0.0056411  0.00560971 0.00555301 0.

NameError: name 'positive_targets_ids' is not defined